# import package

In [ ]:
import pandas as pd
import numpy as np
import os
import math
from scipy.stats import t
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from datetime import date
from pandas.tseries.offsets import MonthEnd
import datetime
from datetime import timedelta
import pyarrow.parquet as pq

# function

In [ ]:
def Fraud_import(df_fraud_path):
    df_fraud = pd.read_excel(df_fraud_path,sheet_name='รวม NPLs')
    df_fraud.rename(columns={'รหัสลูกค้า':'CustomerRefID'},inplace=True)
    df_fraud['CustomerRefID'] = df_fraud['CustomerRefID'].fillna('').astype(str).apply(lambda x: x.zfill(7))
    df_fraud=df_fraud[df_fraud['คดีอาญา']=='Yes']
    fraud_list=df_fraud['CustomerRefID'].unique()
    return fraud_list

def fraud_filter(df,fraud_list):
    df.rename(columns={'Portfolio':'RequestID_Char'},inplace=True)
    df['RatingY']=df['RatingDate'].dt.year
    df_filtered = df[~df['CustomerRefID'].isin(fraud_list)]
    return df_filtered

def change_criteria(datamerge_dict1):
    #เฉพาะ composite
    df_com=datamerge_dict1['Composite']
    df_com = df_com.reset_index(drop=True)
    # หาร10 คะแนนเก่า
    df_com.loc[df_com['CompositeScore']>0,'CompositeScore_New']=df_com['CompositeScore']
    df_com.loc[df_com['CompositeScore']==0,'CompositeScore_New']=df_com["BusinessScore"]+df_com["IndustryScore"]+df_com["adjFinancialScore"]
    df_com.loc[df_com['CompositeScore_New']>100,'CompositeScore_New']=df_com['CompositeScore_New']/10
    #ใช้เกณฑ์ใหม่
    df_com=composite_criteria_change(df_com)
    return df_com


#แปลงเกณฑ์ composite
def composite_criteria_change(df):
    df['new_CompositeRate'] = None
    # Apply the conditions
    mask = df['RequestID_Char'] == 'C'
    df.loc[mask & (df['CompositeScore_New'] >= 92), 'new_CompositeRate'] = 'A'
    df.loc[mask & (df['CompositeScore_New'] >= 89) & (df['CompositeScore_New'] < 92), 'new_CompositeRate'] = 'B1'
    df.loc[mask &(df['CompositeScore_New'] >= 86) & (df['CompositeScore_New'] < 89), 'new_CompositeRate'] = 'B2'
    df.loc[mask &(df['CompositeScore_New'] >= 81) & (df['CompositeScore_New'] < 86), 'new_CompositeRate'] = 'B3'
    df.loc[mask &(df['CompositeScore_New'] >= 78) & (df['CompositeScore_New'] < 81), 'new_CompositeRate'] = 'B4'
    df.loc[mask &(df['CompositeScore_New'] >= 74) & (df['CompositeScore_New'] < 78), 'new_CompositeRate'] = 'C1'
    df.loc[mask &(df['CompositeScore_New'] >= 70) & (df['CompositeScore_New'] < 74), 'new_CompositeRate'] = 'C2'
    df.loc[mask & (df['CompositeScore_New'] < 70), 'new_CompositeRate'] = 'C3'
    
    mask_m = df['RequestID_Char'] == 'M'
    df.loc[mask_m & (df['CompositeScore_New'] >= 96.323), 'new_CompositeRate'] = 'A'
    df.loc[mask_m &(df['CompositeScore_New'] >= 92.546) & (df['CompositeScore_New'] < 96.323), 'new_CompositeRate'] = 'B1'
    df.loc[mask_m &(df['CompositeScore_New'] >= 88.769) & (df['CompositeScore_New'] < 92.546), 'new_CompositeRate'] = 'B2'
    df.loc[mask_m &(df['CompositeScore_New'] >= 84.991) & (df['CompositeScore_New'] < 88.769), 'new_CompositeRate'] = 'B3'
    df.loc[mask_m &(df['CompositeScore_New'] >= 81.214) & (df['CompositeScore_New'] < 84.991), 'new_CompositeRate'] = 'B4'
    df.loc[mask_m &(df['CompositeScore_New'] >= 77.437) & (df['CompositeScore_New'] <= 81.214), 'new_CompositeRate'] = 'C1'
    df.loc[mask_m &(df['CompositeScore_New'] >= 73.660) & (df['CompositeScore_New'] <= 77.437), 'new_CompositeRate'] = 'C2'
    df.loc[mask_m &(df['CompositeScore_New'] < 73.660), 'new_CompositeRate'] = 'C3'
    
    mask_r = df['RequestID_Char'] == 'R'
    df.loc[mask_r &(df['CompositeScore_New'] >= 93), 'new_CompositeRate'] = 'A'
    df.loc[mask_r &(df['CompositeScore_New'] >= 86) & (df['CompositeScore_New'] < 93), 'new_CompositeRate'] = 'B1'
    df.loc[mask_r &(df['CompositeScore_New'] >= 80) & (df['CompositeScore_New'] < 86), 'new_CompositeRate'] = 'B2'
    df.loc[mask_r &(df['CompositeScore_New'] >= 72) & (df['CompositeScore_New'] < 80), 'new_CompositeRate'] = 'B3'
    df.loc[mask_r &(df['CompositeScore_New'] >= 61) & (df['CompositeScore_New'] < 72), 'new_CompositeRate'] = 'B4'
    df.loc[mask_r &(df['CompositeScore_New'] >= 45) & (df['CompositeScore_New'] < 61), 'new_CompositeRate'] = 'C1'
    df.loc[mask_r &(df['CompositeScore_New'] >= 35) & (df['CompositeScore_New'] < 45), 'new_CompositeRate'] = 'C2'
    df.loc[mask_r &(df['CompositeScore_New'] < 35), 'new_CompositeRate'] = 'C3'

    mask_p = df['RequestID_Char'] == 'P'
    df.loc[mask_p &(df['CompositeScore_New'] >= 83), 'new_CompositeRate'] = 'A'
    df.loc[mask_p &(df['CompositeScore_New'] >= 78) & (df['CompositeScore_New'] < 83), 'new_CompositeRate'] = 'B1'
    df.loc[mask_p &(df['CompositeScore_New'] >= 70) & (df['CompositeScore_New'] < 78), 'new_CompositeRate'] = 'B2'
    df.loc[mask_p &(df['CompositeScore_New'] >= 65) & (df['CompositeScore_New'] < 70), 'new_CompositeRate'] = 'B3'
    df.loc[mask_p &(df['CompositeScore_New'] >= 61) & (df['CompositeScore_New'] < 65), 'new_CompositeRate'] = 'B4'
    df.loc[mask_p &(df['CompositeScore_New'] >= 58) & (df['CompositeScore_New'] < 61), 'new_CompositeRate'] = 'C1'
    df.loc[mask_p &(df['CompositeScore_New'] >= 52) & (df['CompositeScore_New'] < 58), 'new_CompositeRate'] = 'C2'
    df.loc[mask_p &(df['CompositeScore_New'] < 52), 'new_CompositeRate'] = 'C3'  
    return df

def IV_test(df, variable):
    #'ExistingDefaultFlag','DefaultFlag12M','adjCompositeScore','adjCompositeRate'
    if variable in ['adjCompositeRate','adjCompositeScore']:
        
        # 1. Drop rows where flag is 1
        df = df.drop(df[df['ExistingDefaultFlag'] == 1].index)
        
        # 2. Check if the required columns exist
        required_columns = ['DefaultFlag12M', 'RequestID', variable]
        for col in required_columns:
            if col not in df.columns:
                raise ValueError(f"Column '{col}' not found in the dataframe.")
        
        # 3. Create 'good_bad' column
        df['good_bad'] = np.where((df['DefaultFlag12M'] == 0), 'good', 'bad')
        
        # 4. Create pivot table by 'CustomerRefID'
        try:
            good_bad_tables = pd.pivot_table(df, values='CustomerRefID', index=[variable], columns='good_bad', aggfunc='count', fill_value=0)
        except Exception as e:
            print(f"Error in creating pivot table: {e}")
            return None, None
        
        # Check if 'good' and 'bad' columns exist in the pivot table
        if 'bad' not in good_bad_tables or 'good' not in good_bad_tables:
            return good_bad_tables, good_bad_tables
    
        table = good_bad_tables.copy()
        if variable=='adjCompositeRate' :
            table.sort_values(variable,ascending=False,inplace=True)
        else:table.sort_values(variable,ascending=True,inplace=True)
        
        # Calculate ROC KS
        table['cumugood'] = table['good'].cumsum() / table['good'].sum()
        table['cumubad'] = table['bad'].cumsum() / table['bad'].sum()
        table['%bad']= 100*table['bad']/(table['good']+table['bad'])
        
        
        table['ROC'] = (table['cumugood'] - table['cumugood'].shift(1, fill_value = 0)) * \
                    (table['cumubad'] + table['cumubad'].shift(1, fill_value = 0)) * 0.5
        table['KS'] = abs(table['cumugood'] - table['cumubad'])
        table['AUC (sum ROC)'] = table['ROC'].sum()
        table['AR'] = (table['AUC (sum ROC)'] * 2) - 1
        
        # Create 'Distribution'
        table['Distribution bad (DB)'] = (table['bad'] / table['bad'].sum())
        table['Distribution good (GB)'] = (table['good'] / table['good'].sum())

        # Calculate WOE IV
        table['WOE'] = np.log(table['Distribution good (GB)'] / table['Distribution bad (DB)'])
        table['DG - DB'] = table['Distribution good (GB)'] - table['Distribution bad (DB)']
        table['IV'] = (table['DG - DB'] * table['WOE'])
        table['sum_IV'] = table[table['IV'] != math.inf]['IV'].sum()
        
        # Calculate KS
        table['max_KS']=max(table['KS'] )
        max_KS=max(table['KS'] )
        totalbad=table['bad'].sum()
        totalgood=table['good'].sum()
        # sqrt((m+n)/(m*n))
        z=np.sqrt((totalbad+totalgood)/(totalbad*totalgood))
        critical_D_05 = z*1.36
        critical_D_01 =z*1.22
        table['Confidential Level 90'] =np.where(max_KS > critical_D_01, 'Reject H0','Fail to Reject H0' )
        table['Confidential Level 95'] =np.where(max_KS > critical_D_05, 'Reject H0','Fail to Reject H0')
        table['Order'] = table['%bad'].is_monotonic_decreasing
        table.drop(columns=['Distribution bad (DB)','Distribution good (GB)','DG - DB'],inplace=True)
        return table, good_bad_tables
    else:
        return None,None

def iv_loop(df_com,file_path):
    good_bad_dict={}
    variable_list=df_com.columns.to_list()
    with pd.ExcelWriter(file_path) as writer:
        for variable in variable_list :
            pivot_table,good_bad_tables=IV_test(df_com, variable)
            if pivot_table is not None:
                styled_df=Apply_style(pivot_table)
                styled_df.to_excel(writer, sheet_name=variable)
                good_bad_dict[variable]=good_bad_tables
 
    return good_bad_dict


    

def all_test(df_com,word,folder_path):
    model_dict={}
    model_list=df_com['RequestID_Char'].unique()
    for i in model_list:
        model_dict[i]=df_com[df_com['RequestID_Char']==i]
        print('port',i,'มีจำนวนทั้งหมด: ',len(df_com[df_com['RequestID_Char']==i]))
        output_path=os.path.join(folder_path,'Output')

    #  composite level 
    for model,df in model_dict.items():
        print('composite level portfolio',model)
        good_bad_model_dict={}
        #IV
        output_path=os.path.join(folder_path,'Output','IV_test',model)
        if not os.path.isdir(output_path):
            os.makedirs(output_path)
        file_path=os.path.join(output_path,f'{word}_{model}_IV_KS_AUC_result_composite.xlsx')
        good_bad_model_dict=iv_loop(df,file_path)
    
        #Homogeneity
        output_path=os.path.join(folder_path,'Output','Homogeneity_test',model)
        if not os.path.isdir(output_path):
            os.makedirs(output_path)
        file_path=os.path.join(output_path,f'{word}_{model}_Homogeneity_result_composite.xlsx')
        Homogeneity_loop(good_bad_model_dict,file_path)

        #PSI
        output_path=os.path.join(folder_path,'Output','PSI_test',model)
        if not os.path.isdir(output_path):
            os.makedirs(output_path)
        file_path=os.path.join(output_path,f'{word}_{model}_PSI_result_composite.xlsx')
        PSI_loop(df,file_path)
      
def PSI_loop(df,file_path): 
    with pd.ExcelWriter(file_path) as writer:
        for var in df.columns.to_list():
            if var in ['CompositeRate','adjCompositeRate']:
                x=df.reset_index()
                if var!='adjCompositeScore' and var != 'CompositeScore':
                    x.sort_values(var, ascending=False,inplace=True)
                else: x.sort_values(var, ascending=True,inplace=True)
                psi_df=PSI_test(x,var)
                styled_df = psi_df.style.map(highlight_PSI, subset=['PSI'])
                styled_df.to_excel(writer, sheet_name=var, index=False)
                
def Homogeneity_loop(good_bad_variable_dict,file_path): 
    with pd.ExcelWriter(file_path) as writer:
        for var,df in good_bad_variable_dict.items():
            if var in ['CompositeRate','adjCompositeRate']:
                x=df.reset_index()
                if var!='adjCompositeScore' and var != 'CompositeScore':
                    x.sort_values(var, ascending=False,inplace=True)
                else: x.sort_values(var, ascending=True,inplace=True)
                Homogeneity_df=Homogeneity(x)
                Homogeneity_df.reset_index(drop=True).to_excel(writer, sheet_name=var, index=False)
    

def Homogeneity(df):
    table = df.copy()
    if 'bad' not in table or 'good' not in table:
        return table
    table['N']=table['bad']+table['good']
    table['N_shift']=table['N'].shift(1)

    table['badRate(X1)'] = table['bad'] / table['N']
    table['badshift(X2)']=table['badRate(X1)'].shift(1)
    
    table['ABS(X1-X2)'] = abs(table['badRate(X1)']- table['badshift(X2)'])
    table['SQRT(X1(1-X1)/n1 + X2(1-X2)/n2)']=np.sqrt( table['badRate(X1)']*(1-table['badRate(X1)'])/table['N'] + table['badshift(X2)']*(1-table['badshift(X2)'])/table['N_shift'] )
    table['t-stat']=table['ABS(X1-X2)']/table['SQRT(X1(1-X1)/n1 + X2(1-X2)/n2)']
    df=len(table)-1
    table['p-value'] = t.sf(table['t-stat'], df)
    table['CL 90%'] = np.where(table['p-value'] > 0.1,'Fail to Reject H0', (np.where(table['p-value'].isnull(),'','Reject H0')))
    table['CL 95%'] = np.where(table['p-value'] > 0.05,'Fail to Reject H0', (np.where(table['p-value'].isnull(),'','Reject H0')))
    return table


def import_factor(new,old):
    new_df=pd.read_parquet(new)
    old_df=pd.read_parquet(old)
    old_df=old_df[old_df['ScoreModel']=='New62']
    old_df['Template']='Old'
    return new_df,old_df
    
def prepare_factor_df(df_corporate,fi_new_df,fi_old_df,retail):
    # filter
    def keep_lastest_ratingdate(df):
        df['RatingDate'] = pd.to_datetime(df['RatingDate'])
        df.sort_values(by='RatingDate', ascending=False, inplace=True)
        
        df['MaxRatingDate'] = df.groupby('RequestID')['RatingDate'].transform('max')
        df_max_rating_date = df[df['RatingDate'] == df['MaxRatingDate']].copy()
        df_max_rating_date.drop(columns=['MaxRatingDate'], inplace=True)
        df_max_rating_date.reset_index(drop=True, inplace=True)
        
        return df_max_rating_date
    fi_new_fil_df=keep_lastest_ratingdate(fi_new_df)
    
    # find dup
    set1=set(fi_new_df['RequestID'].unique())
    set3=set(fi_old_df['RequestID'].unique())
    old_new_dup=set.intersection(set1, set3)
    
    #merge
    fi_old_fil_df=fi_old_df[~fi_old_df['RequestID'].isin(old_new_dup)]
    fi_df = pd.concat([fi_new_fil_df, fi_old_fil_df])
    fi_df3 = fi_df.drop(columns=['RatingYear', 'RequestDate', 'WorkFlowStepName', 'CustomerRefID', 'ScoreModel', 'FlowModelNew62'])
    if retail=='Y':
        fi_df3 = fi_df3.drop(columns=['No','Status', 'CustomerID','CustomerName','BusinessSize','BusinessCodeMOC','BusinessCodeISIC',
'LastApprovalDate', 'FlowModelExisting','RequestCustType','RegistrationID','Model','HasFinancialStatement',
'RevenueEstimate', 'ApprovalLimit','CustType', 'JuristicType','EXIMCustDate','StartDate','BusinessAge'])
        
    fi_df3['RatingDate'] =pd.to_datetime(fi_df3['RatingDate'])
    fi_df3 = fi_df3.sort_values('RatingDate',ascending=False).drop_duplicates(subset='RequestID').drop(columns=['RatingDate'])
    x = df_corporate.merge(fi_df3, on='RequestID', how='left')
    
    def Extract_yearmonth(x):
        if isinstance(x['RequestID'], str)  :
            if x['RequestID'][0] in ['E','N'] :
                return int(str(x['RatingDate'].year)+ str(x['RatingDate'].month).zfill(2)) ,'Old'
            else:
                return int(str(x['RatingDate'].year)+ str(x['RatingDate'].month).zfill(2)) ,'New'
        
    x[['YearMonth', 'ModelUse']]=x.apply(lambda row: pd.Series(Extract_yearmonth(row)), axis=1)  
    x.fillna({'Template': 'New'}, inplace=True)

    def new_Model(x):
        if isinstance(x['Model'], str):
            if x['Model'] in ['C','S','H','F']:
                return 'SE'
            elif x['Model'] in ['TS','TC']:
                return 'TR'
            else: return 'MA'
    x['new_Model']=x.apply(lambda row: pd.Series(new_Model(row)), axis=1)
    
    return x

def factor_prepare_factor_df(df_corporate,df_sme,df_retail,fi_new_df,fi_old_df,bus_new_df,bus_old_df,bussme_new_df,bussme_old_df,ind_new_df,ind_old_df,BX17_df):
    cor_fi=prepare_factor_df(df_corporate,fi_new_df,fi_old_df,'n')
    cor_bus=prepare_factor_df(df_corporate,bus_new_df,bus_old_df,'n')
    cor_in=prepare_factor_df(df_corporate,ind_new_df,ind_old_df,'n')

    sme_fi=prepare_factor_df(df_sme,fi_new_df,fi_old_df,'n')
    sme_bus=prepare_factor_df(df_sme,bussme_new_df,bussme_old_df,'n')
    sme_ind=prepare_factor_df(df_sme,ind_new_df,ind_old_df,'n')

    retail_fi=prepare_factor_df(df_retail,fi_new_df,fi_old_df,'Y')
    retail_bus=prepare_factor_df(df_retail,bus_new_df,bus_old_df,'Y')
    retail_ind=prepare_factor_df(df_retail,ind_new_df,ind_old_df,'Y')

    cor_bus=cor_bus.merge(BX17_df[['RequestID',  'BX17_Sc', 'BX17_MaxSc']],on='RequestID',how='left')
    cor_bus=cor_bus[ (cor_bus['YearMonth']!= 202308) & (cor_bus['YearMonth']!= 202309)  ]

    return cor_fi,cor_bus,cor_in,sme_fi,sme_bus,sme_ind,retail_fi,retail_bus,retail_ind


def import_BX17(directory):
    import glob
    import os
    search_pattern = os.path.join(directory, '**/CRSSCORERESULT*/ScoreResult_Detail.csv')
    file_paths = glob.glob(search_pattern, recursive=True)
    BX17_df = pd.DataFrame(columns=['RequestID',  'TopicVar', 'TopicScore', 'TopicMaxScore'])
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        filtered_df = df[(df['TopicVar'] == 'BX17')][['RequestID',  'TopicVar', 'TopicScore', 'TopicMaxScore']]
        BX17_df = pd.concat([BX17_df, filtered_df], ignore_index=True)
    
    BX17_df.rename(columns={'TopicScore':'BX17_Sc','TopicMaxScore':'BX17_MaxSc'},inplace=True)
    BX17_df.drop_duplicates(inplace=True)
    return BX17_df
    

def run_all_factor (cor_fi,cor_bus,cor_in,sme_fi,sme_bus,sme_ind,retail_fi,retail_bus,retail_ind):
    def cor_NetProfitMargin(x):
        if x['ModelUse']=='Old' and x['Template']=='Old' :
            if  x['NetProfitMargin_Sc'] == 60:
                return  '1 x<=4'
            elif x['NetProfitMargin_Sc'] == 75:
                return '2 4<x<=8'
            elif x['NetProfitMargin_Sc']==87:
                return  '3 x>8'
            else: return  None
                
        elif  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['NetProfitMargin_Sc'] <=4 :
                return  '1 x<=4'
            elif x['NetProfitMargin_Sc'] > 4 and x['NetProfitMargin_Sc'] <=8:
                return '2 4<x<=8'
            elif x['NetProfitMargin_Sc']> 8:
                return  '3 x>8'
            else: return  None     
        else :   
            if  x['NetProfitMargin_Sc'] == 5.97:
                return  '1 x<=4'
            elif x['NetProfitMargin_Sc'] == 7.46:
                return  '2 4<x<=8'
            elif x['NetProfitMargin_Sc']==8.65:
                return  '3 x>8'
           
    def cor_DERatio(x):
        if x['DERatio_MaxSc']!=10:
            if  x['ModelUse']=='Old' and x['Template']=='New' :
                if x['DERatio_Sc'] <=2:
                    return  '1 x<=2'
                elif x['DERatio_Sc'] > 2 and x['DERatio_Sc'] <=6:
                    return '2 2<x<=6'
                elif x['DERatio_Sc']> 6:
                    return  '3 x>6'
                else: return  None
                    
            elif x['ModelUse']=='Old' and x['Template']=='Old' :
                if  x['DERatio_Sc'] == 60:
                    return  '1 x<=2'
                elif x['DERatio_Sc'] == 63:
                    return  '2 2<x<=6'
                elif x['DERatio_Sc']==66:
                    return   '3 x>6'
                else: return  None      
            else :   
                if  x['DERatio_Sc'] == 5.97:
                    return  '1 x<=2'
                elif x['DERatio_Sc'] == 6.27:
                    return   '2 2<x<=6'
                elif x['DERatio_Sc']== 6.57:
                    return   '3 x>6'
                else: return  None

    def cor_TotalDebtsToSales(x):
        if x['ModelUse']=='Old' and x['Template']=='New' :
            if x['TotalDebtsToSales_Sc'] <=2:
                return  '1 x<=2'
            elif x['TotalDebtsToSales_Sc'] > 2 and x['TotalDebtsToSales_Sc'] <=4:
                return '2 2<x<=4'
            elif x['TotalDebtsToSales_Sc'] > 4 and x['TotalDebtsToSales_Sc'] <=5:
                return  '3 4<x<=5'
            elif x['TotalDebtsToSales_Sc']>5:
                return   '4 x>5'
            else: return  None
                
        elif x['ModelUse']=='Old' and x['Template']=='Old' :
            if  x['TotalDebtsToSales_Sc'] == 51:
                return  '1 x<=2'
            elif x['TotalDebtsToSales_Sc'] == 60:
                return  '2 2<x<=4'
            elif x['TotalDebtsToSales_Sc']==72:
                return   '3 4<x<=5'
            elif x['TotalDebtsToSales_Sc']==84:
                return   '4 x>5'
            else: return  None
                
        else :   
            if  x['TotalDebtsToSales_Sc'] == 5.07:
                return  '1 x<=2'
            elif x['TotalDebtsToSales_Sc'] == 5.97:
                return  '2 2<x<=4'
            elif x['TotalDebtsToSales_Sc']==7.16:
                return   '3 4<x<=5'
            elif x['TotalDebtsToSales_Sc']==8.36:
                return   '4 x>5'
            else: return  None

    def cor_CurrentRatio(x):
        if x['CurrentRatio_MaxSc']!=10:
            if  x['ModelUse']=='Old' and x['Template']=='New' :
                if x['CurrentRatio_Sc'] ==0:
                    return  '1 x=0'
                elif x['CurrentRatio_Sc'] > 0 and x['CurrentRatio_Sc'] <=2:
                    return '2 0<x<=2'
                elif x['CurrentRatio_Sc']>2:
                    return   '3 x>2'
                else: return  None
                    
            elif  x['ModelUse']=='Old' and x['Template']=='Old' :
                if  x['CurrentRatio_Sc'] == 54:
                    return  '1 x=0'
                elif x['CurrentRatio_Sc'] == 72:
                    return  '2 0<x<=2'
                elif x['CurrentRatio_Sc']==87:
                    return   '3 x>2'
                else: return  None
                    
            else :   
                if  x['CurrentRatio_Sc'] <= 5.37:
                    return  '1 x=0'
                elif x['CurrentRatio_Sc'] ==7.16 :
                    return  '2 0<x<=2'
                elif x['CurrentRatio_Sc']>7.16:
                    return   '3 x>2'
                else: return  None
    def cor_ReceivableTurnover(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['ReceivableTurnover_Sc'] ==0:
                return  '1 x=0'
            elif x['ReceivableTurnover_Sc'] >0:
                return '2 x>0'
            else: return  None
        elif  x['ModelUse']=='Old' and x['Template']=='Old' :
            if x['ReceivableTurnover_Sc'] ==45:
                return  '1 x=0'
            elif x['ReceivableTurnover_Sc']>45:
                return '2 x>0'
            else: return  None
        else:
            if x['ReceivableTurnover_Sc'] ==4.48:
                return  '1 x=0'
            elif x['ReceivableTurnover_Sc'] >4.48:
                return '2 x>0'
            else: return  None
    def cor_InventoryTurnover(x):
        if x['ModelUse']=='Old' and x['Template']=='New' :
            if x['InventoryTurnover_Sc'] == 0:
                return  '1 x=0'
            elif x['InventoryTurnover_Sc'] > 0 and x['InventoryTurnover_Sc'] <=2:
                return '2 0<x<=2'
            elif x['InventoryTurnover_Sc']> 2:
                return  '3 x>2'    
            else: return  None
        else:
            if  x['InventoryTurnover_Sc'] == 5.97:
                return  '1 x=0'
            elif x['InventoryTurnover_Sc'] == 6.27:
                return '2 0<x<=2'
            elif x['InventoryTurnover_Sc']==6.57:
                return  '3 x>2'
            else: return  None
    def cor_TotalAssetTurnover(x):
        if x['TotalAssetTurnover_MaxSc']!=12:
            if  x['ModelUse']=='Old' and x['Template']=='New' :
                if x['TotalAssetTurnover_Sc'] <=2 :
                    return  '1 x<=2'
                elif x['TotalAssetTurnover_Sc'] >2 and x['TotalAssetTurnover_Sc'] <=4:
                    return '2 2<x<=4'
                elif x['TotalAssetTurnover_Sc'] >4:
                    return '3 x>4'
                else: return  None
            elif  x['ModelUse']=='Old' and x['Template']=='Old' :
                if x['TotalAssetTurnover_Sc'] ==60 :
                    return  '1 x<=2'
                elif x['TotalAssetTurnover_Sc'] ==63:
                    return '2 2<x<=4'
                elif x['TotalAssetTurnover_Sc'] ==66:
                    return '3 x>4'
                else: return  None
            else :
                if x['TotalAssetTurnover_Sc'] ==5.97 :
                    return  '1 x<=2'
                elif x['TotalAssetTurnover_Sc'] ==6.27:
                    return '2 2<x<=4'
                elif x['TotalAssetTurnover_Sc']==6.57:
                    return '3 x>4'
                else: return  None
    def cor_DSCR(x):
        if x['DSCR_MaxSc']!=10:
            if  x['ModelUse']=='Old' and x['Template']=='New' :
                if x['DSCR_Sc'] <=3 :
                    return  '1 x<=3'
                elif x['DSCR_Sc'] >3 and x['DSCR_Sc'] <=6:
                    return '2 3<x<=6'
                elif x['DSCR_Sc'] >6:
                    return '3 x>6'
                else: return  None
            else  :
                if x['DSCR_Sc'] <=5.67 :
                    return  '1 x<=3'
                elif x['DSCR_Sc'] ==6.87:
                    return '2 3<x<=6'
                elif x['DSCR_Sc'] ==7.76:
                    return '3 x>6'
                else: return  None

    def corporate_finance(x):
        x['new_DSCR']=x.apply( cor_DSCR, axis=1)
        x['new_TotalAssetTurnover']=x.apply( cor_TotalAssetTurnover, axis=1)
        x['new_InventoryTurnover']=x.apply( cor_InventoryTurnover , axis=1)
        x['new_ReceivableTurnover']=x.apply( cor_ReceivableTurnover, axis=1)
        x['new_CurrentRatio']=x.apply( cor_CurrentRatio, axis=1)
        x['new_TotalDebtsToSales']=x.apply( cor_TotalDebtsToSales, axis=1)
        x['new_NetProfitMargin']=x.apply( cor_NetProfitMargin, axis=1)
        x['new_DERatio']=x.apply( cor_DERatio, axis=1)
        
        return x
    cor_fi=corporate_finance(cor_fi)


    # ## Cor business

    def cor_N101(x):
        if x['Template']=='Old':
            if  x['new_Model']=='MA' or x['RequestCustType']=='O':    
                if x['N101_Sc'] ==-6 :
                    return  'a x<=1'
                elif x['N101_Sc'] ==6:
                    return 'b 1<x<=3'
                elif x['N101_Sc'] ==15:
                    return 'c 3<x<=5'
                elif x['N101_Sc'] ==21:
                    return 'd 5<x<=7'
                elif x['N101_Sc'] ==24:
                    return 'e x>7'
                else: return  None            
            else : #36
                if x['N101_Sc'] ==-6 :
                    return  'a x<=1'
                elif x['N101_Sc'] ==9:
                    return 'b 1<x<=3'
                elif x['N101_Sc'] ==21:
                    return 'c 3<x<=5'
                elif x['N101_Sc'] ==30:
                    return 'd 5<x<=7'
                elif x['N101_Sc'] ==36:
                    return 'e x>7'
                else: return  None 
        
        else:
            if  x['new_Model']=='MA':
                if x['RequestCustType']=='N' :#2.5
                    if x['BXX1_Sc']== -0.63:
                        return  'a x<=1'
                    elif x['BXX1_Sc'] ==0.63:
                        return 'b 1<x<=3'
                    elif x['BXX1_Sc'] ==1.56:
                        return 'c 3<x<=5'
                    elif x['BXX1_Sc'] ==2.19:
                        return 'd 5<x<=7'
                    elif x['BXX1_Sc'] ==2.5:
                        return 'e x>7'
                    else: return  None       
                else: #2.35
                    if x['BXX1_Sc'] == -0.59:
                        return  'a x<=1'
                    elif x['BXX1_Sc']==0.59:
                        return 'b 1<x<=3'
                    elif x['BXX1_Sc'] ==1.47:
                        return 'c 3<x<=5'
                    elif x['BXX1_Sc'] ==2.06:
                        return 'd 5<x<=7'
                    elif x['BXX1_Sc'] ==2.35:
                        return 'e x>7'
                    else: return  None   
                        
            elif x['RequestCustType']=='N':#3.56
                if x['BXX1_Sc'] ==-0.59 :
                    return  'a x<=1'
                elif x['BXX1_Sc'] ==0.89:
                    return 'b 1<x<=3'
                elif x['BXX1_Sc'] ==2.08:
                    return 'c 3<x<=5'
                elif x['BXX1_Sc'] ==2.97:
                    return 'd 5<x<=7'
                elif x['BXX1_Sc']==3.56:
                    return 'e x>7'
                else: return  None     
            elif x['RequestCustType']=='O':#2.38
                if x['BXX1_Sc'] == -0.59 :
                    return  'a x<=1'
                elif x['BXX1_Sc'] ==0.59:
                    return 'b 1<x<=3'
                elif x['BXX1_Sc'] ==1.49:
                    return 'c 3<x<=5'
                elif x['BXX1_Sc'] ==2.08:
                    return 'd 5<x<=7'
                elif x['BXX1_Sc'] ==2.38:
                    return 'e x>7'
                else: return  None       

    def cor_N102(x):
        if x['Template']=='Old':
            if x['N102_Sc'] ==-0 :
                return  'a x<=1'
            elif x['N102_Sc'] ==3:
                return 'b 1<=3'
            elif x['N102_Sc'] ==6:
                return 'c 3 x=4'
            elif x['N102_Sc'] ==9:
                return 'd 4 x=5'
            else: return  None 
        else:
            if  x['new_Model']=='MA':
                if x['RequestCustType']=='N' :
                    if x['BXX2_Sc']== 0:
                        return  'a x<=1'
                    elif x['BXX2_Sc'] ==0.31:
                        return 'b 1<=3'
                    elif x['BXX2_Sc'] ==0.63:
                        return  'c 3 x=4'
                    elif x['BXX2_Sc'] ==0.94:
                        return 'd 4 x=5'     
                    else: return  None       
                else: 
                    if x['BXX2_Sc'] == 0:
                        return  'a x<=1'
                    elif x['BXX2_Sc']==0.29:
                        return 'b 1<=3'
                    elif x['BXX2_Sc'] ==0.59:
                        return  'c 3 x=4'
                    elif x['BXX2_Sc'] ==0.88:
                        return 'd 4 x=5'
                    else: return  None                     
            else :
                if x['BXX2_Sc'] == 0 :
                    return  'a x<=1'
                elif x['BXX2_Sc'] ==0.30:
                    return 'b 1<=3'
                elif x['BXX2_Sc'] ==0.59:
                    return  'c 3 x=4'
                elif x['BXX2_Sc'] ==0.89:
                    return 'd 4 x=5'
                else: return  None       

    def cor_N103(x):
        if x['Template']=='Old':
            if  x['new_Model']=='MA' or x['RequestCustType']=='O':    
                if x['N103_Sc'] ==0 :
                    return  'a 1 x=0'
                elif x['N103_Sc'] ==12:
                    return 'b 0<x<2'
                elif x['N103_Sc'] ==18:
                    return 'c x>=2'
                else: return  None            
            else : #
                if x['N103_Sc'] ==0 :
                    return  'a 1 x=0'
                elif x['N103_Sc'] ==24:
                    return 'b 0<x<2'
                elif x['N103_Sc'] ==36:
                    return  'c x>=2'
                else: return  None 
        
        else:
            if  x['new_Model']=='MA' :
                if x['RequestCustType']=='O':  #1.76  
                    if x['BXX3_Sc'] ==0 :
                        return  'a 1 x=0'
                    elif x['BXX3_Sc'] ==1.18:
                        return 'b 0<x<2'
                    elif x['BXX3_Sc'] ==1.76:
                        return 'c x>=2'
                    else: return  None   
                else: #1.87
                    if x['BXX3_Sc'] ==0 :
                        return  'a 1 x=0'
                    elif x['BXX3_Sc'] ==1.25:
                        return 'b 0<x<2'
                    elif x['BXX3_Sc'] ==1.87:
                        return 'c x>=2'
                    else: return  None   
                        
            else :
                if  x['RequestCustType']=='O':#1.78
                    if x['BXX3_Sc'] ==0 :
                        return  'a 1 x=0'
                    elif x['BXX3_Sc'] ==1.19:
                        return 'b 0<x<2'
                    elif x['BXX3_Sc'] ==1.78:
                        return  'c x>=2'
                    else: return  None 
                elif  x['RequestCustType']=='N': #3.56
                    if x['BXX3_Sc'] ==0 :
                        return  'a 1 x=0'
                    elif x['BXX3_Sc'] ==2.38:
                        return 'b 0<x<2'
                    elif x['BXX3_Sc'] ==3.56:
                        return  'c x>=2'
                    else: return  None 
                
    def cor_N104(x):
        if x['Template']=='Old':
            if  x['new_Model']=='MA' :
                if x['RequestCustType']=='O':  #36 
                    if x['N104_Sc'] ==0 :
                        return  'a x<20'
                    elif x['N104_Sc'] ==15:
                        return 'b 20<=x<=50'
                    elif x['N104_Sc'] ==21:
                        return 'c 50<x<=75'
                    elif x['N104_Sc'] ==36:
                        return 'd x>75'
                    else: return  None   
                else: #45
                    if x['N104_Sc'] ==0 :
                        return  'a x<20'
                    elif x['N104_Sc'] ==18:
                        return 'b 20<=x<=50'
                    elif x['N104_Sc'] ==24:
                        return'c 50<x<=75'
                    elif x['N104_Sc'] ==45:
                        return 'd x>75'
                    else: return  None   
            else :#54
                if x['N104_Sc'] == 0 :
                    return  'a x<20'
                elif x['N104_Sc'] == 18:
                    return 'b 20<=x<=50'
                elif x['N104_Sc'] == 27:
                    return 'c 50<x<=75'
                elif x['N104_Sc'] == 54:
                    return 'd x>75'
                else: return  None 
            
        else:
            if  x['new_Model']=='MA' :
                if x['RequestCustType']=='O':  #3.53
                    if x['BXX4_Sc'] ==0 :
                        return  'a x<20'
                    elif x['BXX4_Sc'] ==1.47:
                        return 'b 20<=x<=50'
                    elif x['BXX4_Sc'] ==2.06:
                        return'c 50<x<=75'
                    elif x['BXX4_Sc'] ==3.53:
                        return 'd x>75'
                    else: return  None   
                else: #4.69
                    if x['BXX4_Sc'] ==0 :
                        return  'a x<20'
                    elif x['BXX4_Sc'] ==1.88:
                        return 'b 20<=x<=50'
                    elif x['BXX4_Sc'] ==2.50:
                        return'c 50<x<=75'
                    elif x['BXX4_Sc'] ==4.69:
                        return 'd x>75'
                    else: return  None   
                        
            else :#5.35
                    if x['BXX4_Sc'] ==0 :
                        return  'a x<20'
                    elif x['BXX4_Sc'] ==1.78:
                        return 'b 20<=x<=50'
                    elif x['BXX4_Sc'] ==2.67:
                        return 'c 50<x<=75'
                    elif x['BXX4_Sc'] ==5.35:
                        return 'd x>75'
                    else: return  None 
                        
    def cor_N105(x):
        if x['Template']=='Old':
            if x['RequestCustType']=='O':  #36 
                if x['new_Model']=='MA' :
                    if x['N105_Sc'] ==-6 :
                        return  '1 x<2'
                    elif x['N105_Sc'] ==9:
                        return '2 2<=x<=6'
                    elif x['N105_Sc'] ==21:
                        return '3 7<x<=8'
                    elif x['N105_Sc'] ==36:
                        return '4 x>8'
                    else: return  None   
                else: #45
                    if x['N105_Sc'] ==-6 :
                        return  '1 x<2'
                    elif x['N105_Sc'] ==12:
                        return '2 2<=x<=6'
                    elif x['N105_Sc'] ==27:
                        return '3 7<x<=8'
                    elif x['N105_Sc'] ==45:
                        return '4 x>8'
                    else: return  None   
        else:
            if x['RequestCustType']=='O':  #3.53
                if x['new_Model']=='MA' :
                    if x['BXX5_Sc'] == -0.59 :
                        return  '1 x<2'
                    elif x['BXX5_Sc'] ==0.88:
                        return '2 2<=x<=6'
                    elif x['BXX5_Sc'] ==2.06:
                        return '3 7<x<=8'
                    elif x['BXX5_Sc'] ==3.53:
                        return '4 x>8'
                    else: return  None   
                else: #4.46
                    if x['BXX5_Sc'] == -0.59 :
                        return  '1 x<2'
                    elif x['BXX5_Sc'] ==1.19:
                        return '2 2<=x<=6'
                    elif x['BXX5_Sc'] ==2.67:
                        return '3 7<x<=8'
                    elif x['BXX5_Sc'] ==4.46:
                        return '4 x>8'
                    else: return  None 
    def cor_N106(x):
        if x['Template']=='Old':
            if x['new_Model']=='MA' :
                if  x['RequestCustType']=='O': #36
                    if x['N106_Sc'] ==-6 :
                        return  '1 x<=1'
                    elif x['N106_Sc'] ==12:
                        return '2 1<x<3'
                    elif x['N106_Sc'] ==26:
                        return '3 3<=x<=4'
                    elif x['N106_Sc'] ==36:
                        return '4 x>=5'
                    else: return  None   
                else: #45
                    if x['N106_Sc'] ==-6 :
                        return  '1 x<=1'
                    elif x['N106_Sc'] ==15:
                        return '2 1<x<3'
                    elif x['N106_Sc'] ==33:
                        return '3 3<=x<=4'
                    elif x['N106_Sc'] ==45:
                        return '4 x>=5'
                    else: return  None   
            else:#54
                if x['N106_Sc'] ==-6 :
                    return  '1 x<=1'
                elif x['N106_Sc'] ==18:
                    return '2 1<x<3'
                elif x['N106_Sc'] ==39:
                    return '3 3<=x<=4'
                elif x['N106_Sc'] ==54:
                    return '4 x>=5'
                else: return  None           
        else:
            if x['new_Model']=='MA' :
                if  x['RequestCustType']=='O': #3.53	
                    if x['BXX6_Sc'] ==-0.59 :
                        return  '1 x<=1'
                    elif x['BXX6_Sc'] ==1.18:
                        return '2 1<x<3'
                    elif x['BXX6_Sc'] ==2.65:
                        return '3 3<=x<=4'
                    elif x['BXX6_Sc'] ==3.53	:
                        return '4 x>=5'
                    else: return  None   
                else: #4.69	
                    if x['BXX6_Sc'] ==-0.63 :
                        return  '1 x<=1'
                    elif x['BXX6_Sc'] ==1.56:
                        return '2 1<x<3'
                    elif x['BXX6_Sc'] ==3.44:
                        return '3 3<=x<=4'
                    elif x['BXX6_Sc'] ==4.69:
                        return '4 x>=5'
                    else: return  None   
            else:#5.35
                if x['BXX6_Sc'] ==-0.59 :
                    return  '1 x<=1'
                elif x['BXX6_Sc'] ==1.78:
                    return '2 1<x<3'
                elif x['BXX6_Sc'] ==3.86:
                    return '3 3<=x<=4'
                elif x['BXX6_Sc'] ==5.35:
                    return '4 x>=5'
                else: return  None  

    def N107(x):    
        if x['Template']=='Old':
            if x['N107_Sc'] ==3 :
                return  '1 x=-2'
            elif x['N107_Sc'] ==6:
                return '2 x=0'
            else: return  None     
        
        elif x['BXX7_MaxSc'] in [0.59,0.63] :
            if x['new_Model']=='MA' and  x['RequestCustType']=='N': #0.63
                if x['BXX7_Sc'] == 0.32 :
                    return  '1 x=-2'
                elif x['BXX7_Sc'] ==0.63:
                    return '2 x=0'
                else: return  None   
            else: #0.59
                if x['BXX7_Sc'] == 0.29 :
                    return  '1 x=-2'
                elif x['BXX7_Sc'] ==0.59:
                    return '2 x=0'
                else: return  None   

    def N108(x):
        if x['Template']=='Old': #9
            if x['N108_Sc'] ==3 :
                return  '1 x<30'
            elif x['N108_Sc'] ==6:
                return '2 30<=x<60'
            elif x['N108_Sc'] ==9:
                return '3 x>=60'
            else: return  None          
        
        elif x['BXX8_MaxSc'] in [0.88,0.94,0.89] :        
            if x['new_Model']=='MA' :
                if  x['RequestCustType']=='O': #0.88	
                    if x['BXX8_Sc'] == 0.29 :
                        return  '1 x<30'
                    elif x['BXX8_Sc'] ==0.59:
                        return '2 30<=x<60'
                    elif x['BXX8_Sc'] ==0.88:
                        return '3 x>=60'
                    else: return  None   
                else: #0.94
                    if x['BXX8_Sc'] == 0.31 :
                        return  '1 x<30'
                    elif x['BXX8_Sc'] ==0.63:
                        return '2 30<=x<60'
                    elif x['BXX8_Sc'] == 0.94:
                        return '3 x>=60'
                    else: return  None   
            else:#0.89
                if x['BXX8_Sc'] ==0.30 :
                    return  '1 x<30'
                elif x['BXX8_Sc'] ==0.59:
                    return '2 30<=x<60'
                elif x['BXX8_Sc'] == 0.89:
                    return '3 x>=60'
                else: return  None   

    def N109(x):
        if x['Template']=='Old': #9
            if x['new_Model']=='MA' :
                if x['N109_Sc'] == -3 :
                    return  '1 x<=1'
                elif x['N109_Sc'] ==9:
                    return '2 1<x<=3'
                elif x['N109_Sc'] ==12:
                    return '3 x>3'
                else: return  None   
        else: 
            if x['new_Model']=='MA' :
                if  x['RequestCustType']=='O'and x['BXX9_MaxSc']==1.18: #1.18	
                    if x['BXX9_Sc'] == -0.29 :
                        return  '1 x<=1'
                    elif x['BXX9_Sc'] ==0.88:
                        return '2 1<x<=3'
                    elif x['BXX9_Sc'] ==1.18:
                        return '3 x>3'
                    else: return  None   
                elif x['BXX9_MaxSc']==1.25 : #1.25
                    if x['BXX9_Sc'] == -0.31 :
                        return  '1 x<=1'
                    elif x['BXX9_Sc'] ==0.94:
                        return '2 1<x<=3'
                    elif x['BXX9_Sc'] == 1.25:
                        return '3 x>3'
                    else: return  None     

    def N110(x):
        if x['Template']=='Old': #36
            if x['new_Model']=='MA' :
                if x['N110_Sc'] == 0 :
                    return  '1 x<=2'
                elif x['N110_Sc'] ==18:
                    return '2 2<x<=5'
                elif x['N110_Sc'] ==27:
                    return '3 5<x<=7'
                elif x['N110_Sc'] ==36:
                    return '4 x>7'
                else: return  None   
                    
        elif x['new_Model']=='MA' :
            if x['BX10_MaxSc']in[3.53,3.75]:
                if  x['RequestCustType']=='O': #3.53	
                    if x['BX10_Sc'] == 0 :
                        return  '1 x<=2'
                    elif x['BX10_Sc'] == 1.76:
                        return '2 2<x<=5'
                    elif x['BX10_Sc'] == 2.65:
                        return '3 5<x<=7'
                    elif x['BX10_Sc'] == 3.53:
                        return '4 x>7'
                    else: return  None   
                else: #3.75
                    if x['BX10_Sc'] == 0 :
                        return  '1 x<=2'
                    elif x['BX10_Sc'] == 1.88:
                        return '2 2<x<=5'
                    elif x['BX10_Sc'] == 2.81:
                        return '3 5<x<=7'
                    elif x['BX10_Sc'] ==3.75:
                        return '4 x>7'
                    else: return  None  
                        
    def N111(x):
        if x['Template']=='Old': #9
                if x['N111_Sc'] == 0 :
                    return  '1 x<=1'
                elif x['N111_Sc'] ==9:
                    return '2 1<x<=4'
                elif x['N111_Sc'] ==12:
                    return '3 x=5'
                else: return  None   
        else: #9
            if x['new_Model']=='MA' :
                if  x['RequestCustType']=='O'and x['BX11_MaxSc']==1.18: #1.18	
                    if x['BX11_Sc'] == 0 :
                        return  '1 x<=1'
                    elif x['BX11_Sc'] == 0.88:
                        return '2 1<x<=4'
                    elif x['BX11_Sc'] ==1.18:
                        return '3 x=5'
                    else: return  None   
                elif  x['BX11_MaxSc']==1.25: #1.25
                    if x['BX11_Sc'] == 0 :
                        return  '1 x<=1'
                    elif x['BX11_Sc'] ==0.94:
                        return '2 1<x<=4'
                    elif x['BX11_Sc'] == 1.25:
                        return '3 x=5'
                    else: return  None   
            else:
                if x['BXX9_MaxSc']==1.19 : #TR SE BXX9 #1.19
                    if x['BXX9_Sc'] == 0 :
                        return  '1 x<=1'
                    elif x['BXX9_Sc'] == 0.89:
                        return '2 1<x<=4'
                    elif x['BXX9_Sc'] == 1.19:
                        return '3 x=5'

    def N112(x):
        if x['Template']=='Old': #9
                if x['N112_Sc'] == -3 :
                    return  'a x<=1'
                elif x['N112_Sc'] ==3:
                    return 'b x<=3'
                elif x['N112_Sc'] ==6:
                    return 'c x<=5'
                else: return  None   
        else: #9
            if x['new_Model']=='MA' :
                if x['BX12_MaxSc'] in [0.59,0.63]: #MA old
                    if  x['RequestCustType']=='O': 	
                        if x['BX12_Sc'] == -0.30 :
                            return   'a x<=1'
                        elif x['BX12_Sc'] == 0.30:
                            return 'b x<=3'
                        elif x['BX12_Sc'] ==0.59:
                            return 'c x<=5'
                        else: return  None   
                    else:                            # MA NEW
                        if x['BX12_Sc'] == -0.32 :
                            return   'a x<=1'
                        elif x['BX12_Sc'] == 0.32:
                            return 'b x<=3'
                        elif x['BX12_Sc'] == 0.63:
                            return 'c x<=5'
                        else: return  None   
            else:
                if x['BX10_MaxSc'] in [0.59,0.60] : #TR SE 
                    if x['BX10_Sc'] == -0.30 :
                        return   'a x<=1'
                    elif x['BX10_Sc'] == 0.30:
                        return 'b x<=3'
                    elif x['BX10_Sc']  in [0.59,0.60] :
                        return 'c x<=5'   
    def N113(x):
        if x['Template']=='Old': #9
            if x['new_Model'] in ['SE','TR'] and  x['RequestCustType']=='N':#36
                if x['N113_Sc'] == -9 :
                    return  'a x<=1'
                elif x['N113_Sc'] ==18:
                    return 'b 1<x<=3'
                elif x['N113_Sc'] ==24:
                    return 'c 3<x<5'
                elif x['N113_Sc'] ==36:
                    return 'd x=5'
                else: return  None   
            else:# 27
                if x['N113_Sc'] == -3 :
                    return  'a x<=1'
                elif x['N113_Sc'] ==9:
                    return 'b 1<x<=3'
                elif x['N113_Sc'] ==15:
                    return 'c 3<x<5'
                elif x['N113_Sc'] ==27:
                    return 'd x=5'
                else: return  None 
        else: #9
            if x['new_Model']=='MA' :
                if x['BX13_MaxSc'] in [2.65,2.81]: #MA old
                    if  x['RequestCustType']=='O': 	
                        if x['BX13_Sc'] == -0.89 :
                            return   'a x<=1'
                        elif x['BX13_Sc'] == 0.88:
                            return 'b 1<x<=3'
                        elif x['BX13_Sc'] == 1.47:
                            return 'c 3<x<5'
                        elif x['BX13_Sc'] ==2.65:
                            return 'd x=5'
                        else: return  None   
                    else:                            # MA NEW
                        if x['BX13_Sc'] == -0.94 :
                            return   'a x<=1'
                        elif x['BX13_Sc'] == 0.94:
                            return 'b 1<x<=3'
                        elif x['BX13_Sc'] ==1.56:
                            return 'c 3<x<5'
                        elif x['BX13_Sc'] ==2.81:
                            return 'd x=5'
                        else: return  None   
            
            elif x['RequestCustType']=='O':
                if x['BX11_MaxSc']  ==2.67 : #TR SE 
                    if x['BX11_Sc'] == -0.89 :
                        return   'a x<=1'
                    elif x['BX11_Sc'] == 0.89:
                        return 'b 1<x<=3'
                    elif x['BX11_Sc'] == 1.49 :
                        return 'c 3<x<5'  
                    elif x['BX13_Sc'] ==2.67:
                        return 'd x=5'
            elif x['RequestCustType']=='N':
                if x['BX11_MaxSc'] == 3.56 : #TR SE 
                    if x['BX11_Sc'] == -0.89 :
                        return   'a x<=1'
                    elif x['BX11_Sc'] == 1.78 :
                        return 'b 1<x<=3'
                    elif x['BX11_Sc'] == 2.37 :
                        return 'c 3<x<5'  
                    elif x['BX13_Sc'] ==3.56:
                        return 'd x=5'

    def N114(x):
        if x['Template']=='Old': #9
            if x['N114_Sc'] == 3 :
                return  'a x=-2'
            elif x['N114_Sc'] ==6:
                return 'b x=-1'
            elif x['N114_Sc'] ==9:
                return 'c x=0'
            else: return  None     
        else: #9
            if x['new_Model']=='MA' :
                if x['BX14_MaxSc'] in [0.88,0.94]: #MA old
                    if  x['RequestCustType']=='O': 	
                        if x['BX14_Sc'] == -0.29 :
                            return   'a x=-2'
                        elif x['BX14_Sc'] == 0.59:
                            return 'b x=-1'
                        elif x['BX14_Sc'] == 0.88:
                            return 'c x=0'     
                    else:                            # MA NEW
                        if x['BX14_Sc'] == -0.31 :
                            return   'a x=-2'
                        elif x['BX14_Sc'] == 0.62:
                            return 'b x=-1'
                        elif x['BX14_Sc'] == 0.94:
                            return 'c x=0'
                        else: return  None   
            else:
                if x['BX12_MaxSc']  ==0.89 : #TR SE 
                    if x['BX12_Sc'] == -0.30 :
                        return   'a x=-2'
                    elif x['BX12_Sc'] == 0.59:
                        return 'b x=-1'
                    elif x['BX12_Sc'] == 0.89 :
                        return 'c x=0'  
                        
    def N115(x):
        if x['Template']=='Old': 
            if x['N115_Sc'] == 0 :
                return  'a x<=2'
            elif x['N115_Sc'] ==3:
                return 'b x=3'
            elif x['N115_Sc'] ==6:
                return 'c x>=4'
            else: return  None     
        else: #9
            if x['new_Model']=='MA' :
                if x['BX15_MaxSc'] in [0.59,0.62]: #MA old
                    if  x['RequestCustType']=='O': 	
                        if x['BX15_Sc'] == 0  :
                            return   'a x<=2'
                        elif x['BX15_Sc'] == 0.30:
                            return 'b x=3'
                        elif x['BX15_Sc'] == 0.59:
                            return 'c x>=4'   
                    else:                            # MA NEW
                        if x['BX15_Sc'] == 0 :
                            return   'a x<=2'
                        elif x['BX15_Sc'] == 0.31 :
                            return 'b x=3'
                        elif x['BX15_Sc'] == 0.62:
                            return 'c x>=4'
                        else: return  None   
            elif  x['RequestCustType']=='O': 	
                if x['BX13_MaxSc']  ==0.59 : #TR SE 
                    if x['BX13_Sc'] == 0 :
                        return   'a x<=2'
                    elif x['BX13_Sc'] == 0.30:
                        return 'b x=3'
                    elif x['BX13_Sc'] == 0.59 :
                        return 'c x>=4'  
            elif  x['RequestCustType']=='N': 	
                if x['BX13_MaxSc']  ==0.60 : #TR SE 
                    if x['BX13_Sc'] == 0 :
                        return   'a x<=2'
                    elif x['BX13_Sc'] == 0.30:
                        return 'b x=3'
                    elif x['BX13_Sc'] == 0.60 :
                        return 'c x>=4' 

    def N116(x):
        if x['Template']=='Old': 
            if x['N116_Sc'] == 0 :
                return  'a x<=4'
            elif x['N116_Sc'] ==6:
                return 'b x>4'
            else: return  None     
        
        else: #9
            if x['new_Model']=='MA' :
                if x['BX16_MaxSc'] in [0.59,0.62]: #MA old
                    if  x['RequestCustType']=='O': 	
                        if x['BX16_Sc'] == 0  :
                            return   'a x<=4'
                        elif x['BX16_Sc'] == 0.59:
                            return 'b x>4'
                    else:                            # MA NEW
                        if x['BX16_Sc'] == 0 :
                            return   'a x<=4'
                        elif x['BX16_Sc'] == 0.62 :
                            return 'b x>4'

                        else: return  None   
            elif  x['RequestCustType']=='O': 	
                if x['BX14_MaxSc']  ==0.59 : #TR SE 
                    if x['BX14_Sc'] == 0 :
                        return   'a x<=4'
                    elif x['BX14_Sc'] == 0.59:
                        return 'b x>4'
            elif  x['RequestCustType']=='N': 	
                if x['BX14_MaxSc']  ==0.60 : #TR SE 
                    if x['BX14_Sc'] == 0 :
                        return   'a x<=4'
                    elif x['BX14_Sc'] == 0.60:
                        return 'b x>4'
    def N117(x):
        if x['Template']=='Old': 
            if x['new_Model'] in ['SE','TR'] and  x['RequestCustType']=='N':#24
                if x['N117_Sc'] == 0 :
                    return  'a x=1'
                elif x['N117_Sc'] ==12:
                    return 'b x=2'
                elif x['N117_Sc'] ==24:
                    return 'c x=3'
                else: return  None     
            else:#MA
                if x['N117_Sc'] == 0 :
                    return  'a x=1'
                elif x['N117_Sc'] ==9:
                    return 'b x=2'
                elif x['N117_Sc'] ==18:
                    return 'c x=3'
                else: return  None   
        
        else: #9
            if x['new_Model']=='MA' :
                if x['BX17_MaxSc'] in [1.87,1.76]: #MA old
                    if  x['RequestCustType']=='O': 	
                        if x['BX17_Sc'] == 0  :
                            return  'a x=1'
                        elif x['BX17_Sc'] == 0.88:
                            return 'b x=2'
                        elif x['BX17_Sc'] ==1.76:
                            return 'c x=3'
                    else:                            # MA NEW
                        if x['BX17_Sc'] == 0 :
                            return   'a  x=1'
                        elif x['BX17_Sc'] == 0.94 :
                            return 'b x=2'
                        elif x['BX17_Sc'] ==1.87:
                            return 'c x=3'
                        else: return  None   
            elif  x['RequestCustType']=='O': 	
                if x['BX15_MaxSc']  ==1.78	 : #TR SE 
                    if x['BX15_Sc'] == 0 :
                        return   'a  x=1'
                    elif x['BX15_Sc'] ==0.89:
                        return 'b x=2'
                    elif x['BX15_Sc'] ==1.78:
                        return 'c x=3'
            elif  x['RequestCustType']=='N': 	
                if x['BX15_MaxSc']  ==2.38	 : #TR SE 
                    if x['BX15_Sc'] == 0 :
                        return   'a  x=1'
                    elif x['BX15_Sc'] == 1.19:
                        return 'b x=2'
                    elif x['BX15_Sc'] ==2.38	:
                        return 'c x=3'
                        
    def corporate_business(x):
        x['new_N101']=x.apply(cor_N101, axis=1)
        x['new_N102']=x.apply( cor_N102, axis=1)
        x['new_N103']=x.apply( cor_N103 , axis=1)
        x['new_N104']=x.apply( cor_N104, axis=1)
        x['new_N105']=x.apply( cor_N105, axis=1)
        x['new_N106']=x.apply( cor_N106, axis=1)
        x['new_N107']=x.apply( N107, axis=1) 
        x['N108_Sc']=x[[ 'N108MA_Sc','N108TR_Sc','N108SE_Sc']].bfill(axis=1).iloc[:,0]
        x['N108_MaxSc']=x[[ 'N108MA_MaxSc','N108TR_MaxSc','N108SE_MaxSc']].bfill(axis=1).iloc[:,0]
        x['new_N108']=x.apply( N108, axis=1) 
        x['new_N109']=x.apply( N109, axis=1) 
        x['new_N110']=x.apply( N110, axis=1) 
        x['new_N111']=x.apply( N111, axis=1) 
        x['new_N112']=x.apply( N112, axis=1)
        x['new_N113']=x.apply( N113, axis=1)
        x['new_N114']=x.apply( N114, axis=1)
        x['new_N115']=x.apply( N115, axis=1)
        x['new_N116']=x.apply( N116, axis=1)
        x['new_N117']=x.apply( N117, axis=1)
    
        return x
    cor_bus=corporate_business(cor_bus)

    ##corporate
    # ## industry

    def cor_I101(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['I101_Sc'] <10 :
                return  '1 x<10'
            elif x['I101_Sc'] >=10 and x['I101_Sc'] <13:
                return '2 10<=x<13'
            elif x['I101_Sc'] >=13 and x['I101_Sc'] <14:
                return '3 13<=x<14'
            elif x['I101_Sc'] >=14:
                return '4 x>=14'
            else: return  None
        elif  x['ModelUse']=='New' and x['Template']=='New' :
            if x['I101_Sc'] ==-0.15 :
                return  '1 x<10'
            elif x['I101_Sc']==0.2:
                return '2 10<=x<13'
            elif x['I101_Sc']==0.4	:
                return '3 13<=x<14'
            elif x['I101_Sc'] ==0.96:
                return '4 x>=14'
            else: return  None
        else   :
            if x['I101_Sc'] ==-1.5 :
                return  '1 x<10'
            elif x['I101_Sc']==2:
                return '2 10<=x<13'
            elif x['I101_Sc']==4:
                return '3 13<=x<14'
            elif x['I101_Sc'] ==9.5:
                return '4 x>=14'
            else: return  None
    def cor_I102(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['I102_Sc'] <9 :
                return  '1 x<9'
            elif x['I102_Sc'] >=9 and x['I102_Sc'] <11:
                return '2 9<=x<11'
            elif x['I102_Sc'] >=11 and x['I102_Sc'] <13:
                return '3 11<=x<13'
            elif x['I102_Sc'] >=13:
                return '4 x>=13'
            else: return  None
        elif  x['ModelUse']=='New' and x['Template']=='New' :
            if x['I102_Sc'] == -0.7:
                return  '1 x<9'
            elif x['I102_Sc'] ==0.6:
                return '2 9<=x<11'
            elif x['I102_Sc']==1.26:
                return '3 11<=x<13'
            elif x['I102_Sc'] ==2.86:
                return '4 x>=13'
            else: return  None
        else  :
            if x['I102_Sc'] == -7:
                return  '1 x<9'
            elif x['I102_Sc'] ==6:
                return '2 9<=x<11'
            elif x['I102_Sc']==12.5:
                return '3 11<=x<13'
            elif x['I102_Sc'] ==28.5:
                return '4 x>=13'
            else: return  None
    def cor_I103(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['I103_Sc'] <11 :
                return  '1 x<11'
            elif x['I103_Sc'] >=11 and x['I103_Sc'] <13:
                return '2 11<=x<13'
            elif x['I103_Sc'] >=13 and x['I103_Sc'] <14:
                return '3 13<=x<14'
            elif x['I103_Sc'] >=14:
                return '4 x>=14'
            else: return  None
        elif  x['ModelUse']=='New' and x['Template']=='New' :
            if x['I103_Sc'] ==0 :
                return  '1 x<11'
            elif x['I103_Sc'] ==0.35:
                return '2 11<=x<13'
            elif x['I103_Sc'] ==1.16:
                return '3 13<=x<14'
            elif x['I103_Sc'] ==2.76:
                return '4 x>=14'
            else: return  None
        else :
            if x['I103_Sc'] ==0 :
                return  '1 x<11'
            elif x['I103_Sc'] ==3.5:
                return '2 11<=x<13'
            elif x['I103_Sc'] ==11.5:
                return '3 13<=x<14'
            elif x['I103_Sc'] ==27.5:
                return '4 x>=14'
            else: return  None
    def cor_I104(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['I104_Sc'] <9 :
                return  '1 x<9'
            elif x['I104_Sc'] >=9 and x['I104_Sc'] <11:
                return '2 9<=x<11'
            elif x['I104_Sc'] >=11 and x['I104_Sc'] <12:
                return '3 11<=x<12'
            elif x['I104_Sc'] >=11 and x['I104_Sc'] <13:
                return '4 12<=x<13'
            elif x['I104_Sc'] >=13:
                return '5 x>=13'
            else: return  None
        elif  x['ModelUse']=='New' and x['Template']=='New' :
            if x['I104_Sc'] == 0:
                return  '1 x<9'
            elif x['I104_Sc'] ==0.6:
                return '2 9<=x<11'
            elif x['I104_Sc']==1.26:
                return '3 11<=x<12'
            elif x['I104_Sc']==1.36:
                return '4 12<=x<13'
            elif x['I104_Sc']==1.71:
                return '5 x>=13'
            else: return  None
        else :
            if x['I104_Sc'] == 2.5:
                return  '1 x<9'
            elif x['I104_Sc'] ==6:
                return '2 9<=x<11'
            elif x['I104_Sc']==12.5:
                return '3 11<=x<12'
            elif x['I104_Sc']==13.5:
                return '4 12<=x<13'
            elif x['I104_Sc']==17:
                return '5 x>=13'
            else: return  None
    def cor_I105(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['I105_Sc'] <10 :
                return  '1 x<10'
            elif x['I105_Sc'] >=10 and x['I105_Sc'] <12:
                return '2 10<=x<12'
            elif x['I105_Sc'] >=12 and x['I105_Sc'] <14:
                return '3 12<=x<14'
            elif x['I105_Sc'] >=14:
                return '4 x>=14'
            else: return  None
        elif  x['ModelUse']=='Old' and x['Template']=='Old' :
            if x['I105_Sc'] ==-11.5:
                return  '1 x<10'
            elif x['I105_Sc'] ==8:
                return '2 10<=x<12'
            elif x['I105_Sc'] ==13.5:
                return '3 12<=x<14'
            elif x['I105_Sc'] ==17:
                return '4 x>=14'
            else: return  None
        else  :
            if x['I105_Sc'] ==-1.16 :
                return  '1 x<10'
            elif x['I105_Sc'] ==0.8:
                return '2 10<=x<12'
            elif x['I105_Sc'] ==1.36:
                return '3 12<=x<14'
            elif x['I105_Sc'] ==1.71:
                return '4 x>=14'
            else: return  None

    def corporate_industry(cor_in):
        cor_in['new_I105']=cor_in.apply( cor_I105, axis=1)
        cor_in['new_I104']=cor_in.apply( cor_I104, axis=1) 
        cor_in['new_I103']=cor_in.apply( cor_I103, axis=1)  
        cor_in['new_I102']=cor_in.apply( cor_I102, axis=1)
        cor_in['new_I101']=cor_in.apply( cor_I101, axis=1)
 
        return cor_in
    cor_in=corporate_industry(cor_in)


    # # SME
    # ## business

    def EE101(x):
        if x['new_Model']!='SE' : 
            if x['Template']=='Old': 
                if x['EE101_Sc'] == 38 :
                    return 'a x<=7.5'
                elif x['EE101_Sc'] ==41:
                    return 'b x>7.5'
                else: return  None  
            else:
                if x['EXX1_Sc'] == 3.80 :
                    return  'a x<=7.5'
                elif x['EXX1_Sc'] ==4.10:
                    return 'b x>7.5'
                else: return  None 
        else:
            if x['Template']=='Old': 
                if x['EE101_Sc'] == 47 :
                    return 'a x<=7.5'
                elif x['EE101_Sc'] ==50.5:
                    return 'b x>7.5'
                else: return  None  
            else:
                if x['EXX1_Sc'] == 4.71 :
                    return  'a x<=7.5'
                elif x['EXX1_Sc'] ==5.06:
                    return 'b x>7.5'
                else: return  None   

                
    def EE102(x):
        if x['new_Model']!='SE' : 
            if x['Template']=='Old': 
                if x['EE102_Sc'] == -6 :
                    return 'a x=0'
                elif x['EE102_Sc'] ==26.5:
                    return 'b 0<x<=3'
                elif x['EE102_Sc'] ==41:
                    return 'c x>3'
                else: return  None  
            else:
                if x['EXX2_Sc'] == -0.6 :
                    return  'a x=0'
                elif x['EXX2_Sc'] ==2.65:
                    return 'b 0<x<=3'
                elif x['EXX2_Sc'] ==41:
                    return 'c x>3'
                else: return  None 
        else:
            if x['Template']=='Old': 
                if x['EE102_Sc'] == -7 :
                    return 'a x=0'
                elif x['EE102_Sc'] ==32.5:
                    return 'b 0<x<=3'
                elif x['EE102_Sc'] ==50.5:
                    return 'c x>3'
                else: return  None  
            else:
                if x['EXX2_Sc'] == -0.7 :
                    return  'a x=0'
                elif x['EXX2_Sc'] ==3.25:
                    return 'b 0<x<=3'
                elif x['EE102_Sc'] ==5.06:
                    return 'c x>3'
                else: return  None   
                
                    
    def EE103(x): 
        if x['new_Model']!='SE' : 
            if x['Template']=='Old': 
                if x['EE103_Sc'] == 20.5 :
                    return 'a x=0'
                elif x['EE103_Sc'] ==32:
                    return 'b 0<x<=3'
                elif x['EE103_Sc'] ==52.5:
                    return 'c x>3'
                else: return  None  
            else:
                if x['EXX3_Sc'] ==  2.05:
                    return  'a x=0'
                elif x['EXX3_Sc'] == 3.2:
                    return 'b 0<x<=3'
                elif x['EXX3_Sc'] == 5.25:
                    return 'c x>3'
                else: return  None 
        else:
            if x['Template']=='Old': 
                if x['EE103_Sc'] == 25:
                    return 'a x=0'
                elif x['EE103_Sc'] ==39.5:
                    return 'b 0<x<=3'
                elif x['EE103_Sc'] ==64.5:
                    return 'c x>3'
                else: return  None  
            elif x['new_Model']=='SE' : 
                if x['EXX3_Sc'] == 2.5:
                    return  'a x=0'
                elif x['EXX3_Sc'] == 3.95:
                    return 'b 0<x<=3'
                elif x['EXX3_Sc'] == 6.46:
                    return 'c x>3'
                else: return  None  


    def EE104(x): 
        if x['new_Model']!='SE' : 
            if x['Template']=='Old': 
                if x['EE104_Sc'] == 32 :
                    return 'a x<=1'
                elif x['EE104_Sc'] ==47:
                    return 'b 1<x<=2'
                elif x['EE104_Sc'] ==52.5:
                    return 'c x>2'
                else: return  None  
            else:
                if x['EXX4_Sc'] ==  3.2:
                    return  'a x<=1'
                elif x['EXX4_Sc'] == 4.7:
                    return 'b 1<x<=2'
                elif x['EXX4_Sc'] == 5.25:
                    return 'c x>2'
                else: return  None 
        else:
            if x['Template']=='Old': 
                if x['EE104_Sc'] == 39.5:
                    return 'a x<=1'
                elif x['EE104_Sc'] ==57.5:
                    return 'b 1<x<=2'
                elif x['EE104_Sc'] ==64.5:
                    return 'c x>2'
                else: return  None  
            elif x['new_Model']=='SE' : 
                if x['EXX4_Sc'] ==  3.96:
                    return  'a x<=1'
                elif x['EXX4_Sc'] == 5.76:
                    return 'b 1<x<=2'
                elif x['EXX4_Sc'] == 6.46:
                    return 'c x>2'
                else: return  None   
        
    def EE201(x): 
        if x['new_Model']!='SE' : 
            if x['Template']=='Old': 
                if x['EE201_Sc'] == 14.5 :
                    return 'a x<=1'
                elif x['EE201_Sc'] == 26.5:
                    return 'b 1<x<=2'
                elif x['EE201_Sc'] ==38:
                    return 'c x>2'
                elif x['EE201_Sc'] == 49.5:
                    return 'd x>4'
                else: return  None  
            else:
                if x['EXX5_Sc'] ==  1.45:
                    return  'a x<=1'
                elif x['EXX5_Sc'] == 2.65:
                    return 'b 1<x<=2'
                elif x['EXX5_Sc'] == 3.8:
                    return 'c x>2'
                elif x['EXX5_Sc'] == 4.95:
                    return 'd x>4'
                else: return  None 
        else:
            if x['Template']=='Old': 
                if x['EE201_Sc'] == 18:
                    return 'a x<=1'
                elif x['EE201_Sc'] == 32.5:
                    return 'b 1<x<=2'
                elif x['EE201_Sc'] == 47:
                    return 'c x>2'
                elif x['EE201_Sc'] == 61:
                    return 'd x>4'
                else: return  None  
            elif x['new_Model']=='SE' : 
                if x['EXX5_Sc'] ==  1.80:
                    return  'a x<=1'
                elif x['EXX5_Sc'] == 2.25:
                    return 'b 1<x<=2'
                elif x['EXX5_Sc'] == 4.70:
                    return 'c x>2'
                elif x['EXX5_Sc'] == 6.11:
                    return 'd x>4'
                else: return  None   
    def EE202(x): 
        if x['new_Model']!='SE' : 
            if x['Template']=='Old': 
                if x['EE202_Sc'] == 38 :
                    return 'a x<=1'
                elif x['EE202_Sc'] == 67.5:
                    return 'b x>1'
                else: return  None  
            else:
                if x['EXX6_Sc'] ==  3.8:
                    return  'a x<=1'
                elif x['EXX6_Sc'] == 6.75:
                    return 'b x>1'
                else: return  None 

    def EE203(x): 
        if x['new_Model']!='SE' : 
            if x['Template']=='Old': 
                if x['EE203_Sc'] == 38 :
                    return 'a x<=1'
                elif x['EE203_Sc'] == 41:
                    return 'b 1<x<=2'
                elif x['EE203_Sc'] == 44:
                    return 'c x>2'
                else: return  None  
            else:
                if x['EXX7_Sc'] ==  3.8:
                    return  'a x<=1'
                elif x['EXX7_Sc'] == 4.1:
                    return 'b 1<x<=2'
                elif x['EXX7_Sc'] == 4.4:
                    return 'c x>2'
                else: return  None 
    def EXX6(x): 
        if x['new_Model']=='SE' : 
            if x['Template']=='Old': 
                if x['EE203_Sc'] == 7 :
                    return 'a x<5'
                elif x['EE203_Sc'] ==14.5:
                    return 'b 5<=x<10'
                elif x['EE203_Sc'] == 21.5:
                    return 'c x>=10'
                else: return  None  
            else:
                if x['EXX6_Sc'] ==  0.7:
                    return  'a x<5'
                elif x['EXX6_Sc'] == 1.45:
                    return 'b 5<=x<10'
                elif x['EXX6_Sc'] == 2.15:
                    return 'c x>=10'
                else: return  None 

    def EE204(x): 
        if x['new_Model']!='SE' : 
            if x['Template']=='Old': 
                if x['EE204_Sc'] == 29 :
                    return 'a x<=1'
                elif x['EE204_Sc'] == 38:
                    return 'b  1<x<=4'
                elif x['EE204_Sc'] == 47:
                    return 'c x>4'
                else: return  None  
            else:
                if x['EXX8_Sc'] ==  2.9:
                    return  'a x<=1'
                elif x['EXX8_Sc'] == 3.8:
                    return 'b  1<x<=4'
                elif x['EXX8_Sc'] == 4.7:
                    return 'c x>4'
                else: return  None 
        else:
            if x['Template']=='Old': 
                if x['EE302_Sc'] ==  36:
                    return 'a x<=1'
                elif x['EE302_Sc'] == 47:
                    return 'b  1<x<=4'
                elif x['EE302_Sc'] == 57.5:
                    return 'c x>4'
                else: return  None  
            elif x['new_Model']=='SE' : 
                if x['EXX7_Sc'] ==  3.61:
                    return  'a x<=1'
                elif x['EX10_Sc'] == 4.71:
                    return 'b  1<x<=4'
                elif x['EXX7_Sc'] ==5.76:
                    return 'c x>4'
                else: return  None  
        
    def EE301(x): 
        if x['new_Model']!='SE' : 
            if x['Template']=='Old': 
                if x['EE301_Sc'] == 32 :
                    return 'a x<= -4'
                elif x['EE301_Sc'] == 41:
                    return 'b  -4<x<=2'
                elif x['EE301_Sc'] ==52.5:
                    return 'c x>-2'
                else: return  None  
            else:
                if x['EXX9_Sc'] ==  3.20:
                    return  'a x<= -4'
                elif x['EXX9_Sc'] == 4.10:
                    return 'b  -4<x<=2'
                elif x['EXX9_Sc'] == 5.25:
                    return 'c x>-2'
                else: return  None 
        else:
            if x['Template']=='Old': 
                if x['EE301_Sc'] ==  39.5:
                    return 'a x<= -4'
                elif x['EE301_Sc'] == 50.5:
                    return 'b  -4<x<=2'
                elif x['EE301_Sc'] == 64.5:
                    return 'c x>-2'
                else: return  None  
            elif x['new_Model']=='SE' : 
                if x['EXX8_Sc'] ==  3.95:
                    return  'a x<= -4'
                elif x['EXX8_Sc'] == 5.05:
                    return 'b  -4<x<=2'
                elif x['EXX8_Sc'] == 6.46:
                    return 'c x>-2'
                else: return  None  


    def EE302(x): 
        if x['new_Model']!='SE' : 
            if x['Template']=='Old': 
                if x['EE302_Sc'] == -3 :
                    return 'a x<=-9'
                elif x['EE302_Sc'] == 20.5:
                    return 'b  -9 <x<=-4'
                elif x['EE302_Sc'] ==41:
                    return 'c -4<x<=1'
                elif x['EE302_Sc'] ==52.5:
                    return 'd x>-1'
                else: return  None  
            else:
                if x['EX10_Sc'] ==  -0.3:
                    return  'a x<=-9'
                elif x['EX10_Sc'] == 2.05:
                    return 'b  -9 <x<=-4'
                elif x['EX10_Sc'] == 4.1:
                    return 'c -4<x<=1'
                elif x['EX10_Sc'] == 5.25:
                    return 'd x>-1'
                else: return  None 
        else:
            if x['Template']=='Old': 
                if x['EE302_Sc'] ==  -3.5:
                    return 'a x<=-9'
                elif x['EE302_Sc'] == 25:
                    return 'b  -9 <x<=-4'
                elif x['EE302_Sc'] == 50.5:
                    return 'c -4<x<=1'
                elif x['EE302_Sc'] == 64.5:
                    return 'd x>-1'
                else: return  None  
            elif x['new_Model']=='SE' : 
                if x['EXX9_Sc'] ==  -0.35:
                    return  'a x<=-9'
                elif x['EXX9_Sc'] == 2.5:
                    return 'b  -9 <x<=-4'
                elif x['EXX9_Sc'] == 5.05:
                    return 'c -4<x<=1'
                elif x['EXX9_Sc'] == 6.46:
                    return  'd x>-1'
                else: return  None  

    def sme_business(x):
        x['new_EE101']=x.apply( EE101, axis=1)
        x['new_EE102']=x.apply( EE102, axis=1)
        x['new_EE103']=x.apply( EE103 , axis=1)
        x['new_EE104']=x.apply( EE104, axis=1)
        x['new_EE201']=x.apply( EE201, axis=1)
        x['new_EE202']=x.apply( EE202, axis=1)
        x['new_EE203']=x.apply( EE203, axis=1)
        x['new_EXX6']=x.apply( EXX6, axis=1) 
        x['new_EE204']=x.apply( EE204, axis=1) 
        x['new_EE301']=x.apply( EE301, axis=1) 
        x['new_EE302']=x.apply( EE302, axis=1) 
       
        return x

    sme_bus=sme_business(sme_bus)

    # SME
    # ## finance

    def SME_NetProfitMargin(x):
        if x['ModelUse']=='Old' and x['Template']=='New' :
            if  x['NetProfitMargin_Sc'] <=6:
                return  '1 x<=6'
            elif x['NetProfitMargin_Sc'] >6:
                return '2 x>6'
            else: return  None
        elif x['ModelUse']=='New' and x['Template']=='New' :
            if  x['NetProfitMargin_Sc'] ==4.39:
                return  '1 x<=6'
            elif x['NetProfitMargin_Sc'] ==4.67:
                return '2 x>6'
            else: return  None
        else :
            if  x['NetProfitMargin_Sc'] ==44:
                return  '1 x<=6'
            elif x['NetProfitMargin_Sc'] ==46.8:
                return '2 x>6'
            else: return  None
    def SME_DERatio(x):
        if x['DERatio_MaxSc']!=10:
            if  x['ModelUse']=='Old' and x['Template']=='New' :
                if x['DERatio_Sc'] <=2:
                    return  '1 x<=2'
                elif x['DERatio_Sc'] > 2 and x['DERatio_Sc'] <=6:
                    return '2 2<x<=6'
                elif x['DERatio_Sc']> 6:
                    return  '3 x>6'
                else: return  None
            elif  x['ModelUse']=='New' and x['Template']=='New' :
                if x['DERatio_Sc'] ==4.15:
                    return  '1 x<=2'
                elif x['DERatio_Sc']==4.67:
                    return '2 2<x<=6'
                elif x['DERatio_Sc']==4.91:
                    return  '3 x>6'
                else: return  None
            else   :
                if x['DERatio_Sc'] ==41.6:
                    return  '1 x<=2'
                elif x['DERatio_Sc']==46.8:
                    return '2 2<x<=6'
                elif x['DERatio_Sc']==49.2:
                    return  '3 x>6'
                else: return  None
    def SME_TotalDebtsToSales(x):
        if x['ModelUse']=='Old' and x['Template']=='New' :
            if x['TotalDebtsToSales_Sc'] <=2:
                return  '1 x<=2'
            elif x['TotalDebtsToSales_Sc'] > 2 and x['TotalDebtsToSales_Sc'] <=4:
                return '2 2<x<=4'
            elif x['TotalDebtsToSales_Sc'] > 4 and x['TotalDebtsToSales_Sc'] <=5:
                return  '3 4<x<=5'
            elif x['TotalDebtsToSales_Sc']>5:
                return   '4 x>5'
            else: return  None
        elif x['ModelUse']=='New' and x['Template']=='New' :
            if x['TotalDebtsToSales_Sc'] ==4.15:
                return  '1 x<=2'
            elif x['TotalDebtsToSales_Sc'] ==4.39:
                return '2 2<x<=4'
            elif x['TotalDebtsToSales_Sc'] ==4.67:
                return  '3 4<x<=5'
            elif x['TotalDebtsToSales_Sc']==4.91:
                return   '4 x>5'
            else: return  None
        else  :
            if x['TotalDebtsToSales_Sc'] ==41.6:
                return  '1 x<=2'
            elif x['TotalDebtsToSales_Sc'] ==44:
                return '2 2<x<=4'
            elif x['TotalDebtsToSales_Sc'] ==46.8:
                return  '3 4<x<=5'
            elif x['TotalDebtsToSales_Sc']==49.2:
                return   '4 x>5'
            else: return  None
    def SME_CurrentRatio(x):
        if x['CurrentRatio_MaxSc']!=10:
            if  x['ModelUse']=='Old' and x['Template']=='New' :
                if x['CurrentRatio_Sc']==0:
                    return  '1 x=0'
                elif x['CurrentRatio_Sc'] > 0 and x['CurrentRatio_Sc'] <=2:
                    return '2 0<x<=2'
                elif x['CurrentRatio_Sc']>2:
                    return   '3 x>2'
                else: return  None
            elif  x['ModelUse']=='New' and x['Template']=='New' :
                if x['CurrentRatio_Sc'] ==4.39:
                    return  '1 x=0'
                elif x['CurrentRatio_Sc'] ==4.67:
                    return '2 0<x<=2'
                elif x['CurrentRatio_Sc'] ==4.91:
                    return   '3 x>2'
                else: return  None
            else  :
                if x['CurrentRatio_Sc'] ==44:
                    return  '1 x=0'
                elif x['CurrentRatio_Sc'] ==46.8:
                    return '2 0<x<=2'
                elif x['CurrentRatio_Sc'] ==49.2:
                    return   '3 x>2'
                else: return  None
    def SME_ReceivableTurnover(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['ReceivableTurnover_Sc'] ==0:
                return  '1 x=0'
            elif x['ReceivableTurnover_Sc'] >0:
                return '2 x>0'
            else: return  None
        elif  x['ModelUse']=='New' and x['Template']=='New' :
            if x['ReceivableTurnover_Sc'] ==3.43:
                return  '1 x=0'
            elif x['ReceivableTurnover_Sc'] ==4.91:
                return '2 x>0'
            else: return  None
        else   :
            if x['ReceivableTurnover_Sc'] ==34.4:
                return  '1 x=0'
            elif x['ReceivableTurnover_Sc'] ==49.2:
                return '2 x>0'
            else: return  None
    def SME_InventoryTurnover(x):
        if x['ModelUse']=='Old' and x['Template']=='New' :
            if x['InventoryTurnover_Sc'] == 0:
                return  '1 x=0'
            elif x['InventoryTurnover_Sc'] > 0 and x['InventoryTurnover_Sc'] <=2:
                return '2 0<x<=2'
            elif x['InventoryTurnover_Sc']> 2:
                return  '3 x>2'    
            else: return  None
        elif x['ModelUse']=='New' and x['Template']=='New' :
            if x['InventoryTurnover_Sc'] == 4.39:
                return  '1 x=0'
            elif x['InventoryTurnover_Sc'] ==4.67:
                return '2 0<x<=2'
            elif x['InventoryTurnover_Sc']==4.91:
                return  '3 x>2'    
            else: return  None
    def SME_TotalAssetTurnover(x):
        if x['TotalAssetTurnover_MaxSc']!=12:
            if  x['ModelUse']=='Old' and x['Template']=='New' :
                if x['TotalAssetTurnover_Sc'] ==0 :
                    return  '1 x==0'
                elif x['TotalAssetTurnover_Sc'] >0 and x['TotalAssetTurnover_Sc'] <=4:
                    return '2 0<x<=4'
                elif x['TotalAssetTurnover_Sc'] >4:
                    return '3 x>4'
                else: return  None
            elif  x['ModelUse']=='New' and x['Template']=='New' :
                if x['TotalAssetTurnover_Sc'] ==4.39 :
                    return  '1 x==0'
                elif x['TotalAssetTurnover_Sc'] ==4.67:
                    return '2 0<x<=4'
                elif x['TotalAssetTurnover_Sc'] ==5.15:
                    return '3 x>4'
                else: return  None
            else :
                if x['TotalAssetTurnover_Sc'] ==44 :
                    return  '1 x==0'
                elif x['TotalAssetTurnover_Sc'] ==46.8:
                    return '2 0<x<=4'
                elif x['TotalAssetTurnover_Sc'] ==51.6:
                    return '3 x>4'
                else: return  None
    def SME_DSCR(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['DSCR_Sc'] <=4 :
                return  '1 x<=4'
            elif x['DSCR_Sc'] >4 and x['DSCR_Sc'] <=6:
                return '2 4<x<=6'
            elif x['DSCR_Sc'] >6:
                return '3 x>6'
            else: return  None
        else  :
            if x['DSCR_Sc'] <=4.15 :
                return  '1 x<=4'
            elif x['DSCR_Sc'] ==4.91:
                return '2 4<x<=6'
            elif x['DSCR_Sc'] ==5.63:
                return '3 x>6'
            else: return  None

    def sme_finance(sme_fi):
        sme_fi['new_DSCR']=sme_fi.apply(SME_DSCR , axis=1)
        sme_fi['new_TotalAssetTurnover']=sme_fi.apply(SME_TotalAssetTurnover , axis=1)        
        sme_fi['new_InventoryTurnover']=sme_fi.apply( SME_InventoryTurnover, axis=1)
        sme_fi['new_ReceivableTurnover']=sme_fi.apply( SME_ReceivableTurnover, axis=1)
        sme_fi['new_CurrentRatio']=sme_fi.apply( SME_CurrentRatio, axis=1)
        sme_fi['new_TotalDebtsToSales']=sme_fi.apply( SME_TotalDebtsToSales, axis=1)             
        sme_fi['new_DERatio']=sme_fi.apply( SME_DERatio, axis=1)  
        sme_fi['new_NetProfitMargin']=sme_fi.apply( SME_NetProfitMargin, axis=1)
        return sme_fi
    sme_fi=sme_finance(sme_fi)    

    #SME
    # ## industry

    def SME_I101(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['I101_Sc'] <10 :
                return  '1 x<10'
            elif x['I101_Sc'] >=10 and x['I101_Sc'] <13:
                return '2 10<=x<13'
            elif x['I101_Sc'] >=13 and x['I101_Sc'] <14:
                return '3 13<=x<14'
            elif x['I101_Sc'] >14:
                return '4 x>=14'
            else: return  None
        elif  x['ModelUse']=='New' and x['Template']=='New' :
            if x['I101_Sc'] == -0.15 :
                return  '1 x<10'
            elif x['I101_Sc']==0.2:
                return '2 10<=x<13'
            elif x['I101_Sc']==0.4:
                return '3 13<=x<14'
            elif x['I101_Sc'] ==0.96:
                return '4 x>=14'
            else: return  None
        else   :#old old
            if x['I101_Sc'] == -1.5 :
                return  '1 x<10'
            elif x['I101_Sc']==2:
                return '2 10<=x<13'
            elif x['I101_Sc']==4:
                return '3 13<=x<14'
            elif x['I101_Sc'] ==9.5:
                return '4 x>=14'
            else: return  None
    def SME_I102(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['I102_Sc'] <9 :
                return  '1 x<9'
            elif x['I102_Sc'] >=9 and x['I102_Sc'] <11:
                return '2 9<=x<11'
            elif x['I102_Sc'] >=11 and x['I102_Sc'] <13:
                return '3 11<=x<13'
            elif x['I102_Sc'] >=13:
                return '4 x>=13'
            else: return  None
        elif  x['ModelUse']=='New' and x['Template']=='New' :
            if x['I102_Sc'] == -0.7:
                return  '1 x<9'
            elif x['I102_Sc'] ==0.6:
                return '2 9<=x<11'
            elif x['I102_Sc']==1.26:
                return '3 11<=x<13'
            elif x['I102_Sc'] ==2.86:
                return '4 x>=13'
            else: return  None
        else  :
            if x['I102_Sc'] == -7:
                return  '1 x<9'
            elif x['I102_Sc'] ==6:
                return '2 9<=x<11'
            elif x['I102_Sc']==12.5:
                return '3 11<=x<13'
            elif x['I102_Sc'] ==28.5:
                return '4 x>=13'
            else: return  None
    def SME_I103(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['I103_Sc'] <11 :
                return  '1 x<11'
            elif x['I103_Sc'] >11 and x['I103_Sc'] <=13:
                return '2 11<=x<13'
            elif x['I103_Sc'] >13 and x['I103_Sc'] <14:
                return '3 13<=x<14'
            elif x['I103_Sc'] >14:
                return '4 x>=14'
            else: return  None
        elif  x['ModelUse']=='New' and x['Template']=='New' :
            if x['I103_Sc'] ==0 :
                return  '1 x<11'
            elif x['I103_Sc'] ==0.35:
                return '2 11<=x<13'
            elif x['I103_Sc'] ==1.16:
                return '3 13<=x<14'
            elif x['I103_Sc'] ==2.76:
                return '4 x>=14'
            else: return  None
        else :
            if x['I103_Sc'] ==0 :
                return  '1 x<11'
            elif x['I103_Sc'] ==3.5:
                return '2 11<=x<13'
            elif x['I103_Sc'] ==11.5:
                return '3 13<=x<14'
            elif x['I103_Sc'] ==27.5:
                return '4 x>=14'
            else: return  None
    def SME_I104(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['I104_Sc'] <9 :
                return  '1 x<9'
            elif x['I104_Sc'] >=9 and x['I104_Sc'] <11:
                return '2 9<=x<11'
            elif x['I104_Sc'] >=11 and x['I104_Sc'] <12:
                return '3 11<=x<12'
            elif x['I104_Sc'] >=11 and x['I104_Sc'] <13:
                return '4 12<=x<13'
            elif x['I104_Sc'] >=13:
                return '5 x>=13'
            else: return  None
        elif  x['ModelUse']=='New' and x['Template']=='New' :
            if x['I104_Sc'] == 0.25:
                return  '1 x<9'
            elif x['I104_Sc'] ==0.6:
                return '2 9<=x<11'
            elif x['I104_Sc']==1.26:
                return '3 11<=x<12'
            elif x['I104_Sc']==1.36:
                return '4 12<=x<13'
            elif x['I104_Sc']==1.71:
                return '5 x>=13'
            else: return  None
        else :
            if x['I104_Sc'] == 2.5:
                return  '1 x<9'
            elif x['I104_Sc'] ==6:
                return '2 9<=x<11'
            elif x['I104_Sc']==12.5:
                return '3 11<=x<12'
            elif x['I104_Sc']==13.5:
                return '4 12<=x<13'
            elif x['I104_Sc']==17:
                return '5 x>=13'
            else: return  None
    def SME_I105(x):
        if  x['ModelUse']=='Old' and x['Template']=='New' :
            if x['I105_Sc'] <10 :
                return  '1 x<10'
            elif x['I105_Sc'] >=10 and x['I105_Sc'] <12:
                return '2 10<=x<12'
            elif x['I105_Sc'] >=12 and x['I105_Sc'] <14:
                return '3 12<=x<14'
            elif x['I105_Sc'] >=14:
                return '4 x>=14'
            else: return  None
        elif  x['ModelUse']=='New' and x['Template']=='New' :
            if x['I105_Sc'] ==-1.16 :
                return  '1 x<10'
            elif x['I105_Sc'] ==0.8:
                return '2 10<=x<12'
            elif x['I105_Sc'] ==1.36:
                return '3 12<=x<14'
            elif x['I105_Sc'] ==1.71:
                return '4 x>=14'
            else: return  None
        else   :
            if x['I105_Sc'] ==-11.5:
                return  '1 x<10'
            elif x['I105_Sc'] ==8:
                return '2 10<=x<12'
            elif x['I105_Sc'] ==13.5:
                return '3 12<=x<14'
            elif x['I105_Sc'] ==17:
                return '4 x>=14'
            else: return  None

    def sme_industry(sme_ind):
        sme_ind['new_I105']=sme_ind.apply( SME_I105, axis=1)
        sme_ind['new_I104']=sme_ind.apply( SME_I104, axis=1) 
        sme_ind['new_I103']=sme_ind.apply( SME_I103, axis=1)
        sme_ind['new_I102']=sme_ind.apply( SME_I102, axis=1)
        sme_ind['new_I101']=sme_ind.apply( SME_I101, axis=1)
        return sme_ind
    sme_ind=sme_industry(sme_ind)


    # # Retail
    # ## indus
    
    def Retail_I101(x):
        if x['I101']<10:
            return 'a x<10'
        elif x['I101']>=10 and x['I101']<13:
            return 'b 10<=x<13'
        elif x['I101']>=13 and x['I101']<14:
            return 'c 13<=x<14'
        elif x['I101']>=14:
            return 'd x>14'

    def Retail_I102(x):
        if x['I102']<9:
            return 'a x<9'
        elif x['I102']>=9 and x['I102']<11:
            return 'b 9<=x<11'
        elif x['I102']>=11 and x['I102']<13:
            return 'c 11<=x<13'
        elif x['I102']>=13:
            return 'd x>=13'

    def Retail_I103(x):
        if x['I103']<11:
            return 'a x<11'
        elif x['I103']>=11 and x['I103']<13:
            return 'b 11<=x<13'
        elif x['I103']>=13 and x['I103']<14:
            return 'c 13<=x<14'
        elif x['I103']>=14:
            return 'd x>14'

    def Retail_I104(x):
        if x['I104']<9:
            return 'a x<9'
        elif x['I104']>=9 and x['I104']<11:
            return 'b 9<=x<11'
        elif x['I104']>=11 and x['I104']<12:
            return 'c 11<=x<12'
        elif x['I104']>=12 and x['I104']<13:
            return 'd 12<=x<13'
        elif x['I104']>=13:
            return 'e x>13'


    def Retail_I105(x):
        if x['I105']<10:
            return 'a x<10'
        elif x['I105']>=10 and x['I105']<12:
            return 'b 10<=x<12'
        elif x['I105']>=12 and x['I105']<14:
            return 'c 12<=x<14'
        elif x['I105']>=14:
            return 'd x>14'

    def retail_industry(retail_ind):
        retail_ind['new_I101']=retail_ind.apply(Retail_I101, axis=1)
        retail_ind['new_I102']=retail_ind.apply(Retail_I102, axis=1)
        retail_ind['new_I103']=retail_ind.apply(Retail_I103, axis=1)
        retail_ind['new_I104']=retail_ind.apply(Retail_I104, axis=1)
        retail_ind['new_I105']=retail_ind.apply(Retail_I105, axis=1)
        return retail_ind

    retail_ind=retail_industry(retail_ind)

    # ## FInance
    # ## Business

    def new_NCB7(x):
        if x['RequestCustType']=='N':
            if x['NCB7_Sc'] ==5.25:
                return 3.5
            elif x['NCB7_Sc'] ==10.5:
                return 7
            elif x['NCB7_Sc'] ==15:
                return 10
        else:return  x['NCB7_Sc']
            
    def new_NCB4(x):
        if x['RequestCustType']=='N':
            if x['NCB4_Sc'] ==4:
                return 2
            elif x['NCB4_Sc'] ==8:
                return 4
            elif x['NCB4_Sc'] ==10:
                return 5
        else:return  x['NCB4_Sc']

    def retail_Business(sme_ind):
        sme_ind['new_NCB7']=sme_ind.apply( new_NCB7, axis=1)  
        sme_ind['new_NCB4']=sme_ind.apply( new_NCB4, axis=1)
        return sme_ind

    retail_bus=retail_Business(retail_bus)
    dict_factor={'C':{'cor_fi':cor_fi,'cor_bus':cor_bus,'cor_ind':cor_in},
                 'M':{'sme_fi':sme_fi,'sme_bus':sme_bus,'sme_ind':sme_ind}  ,
                 'R':{'re_fi':retail_fi,'re_bus':retail_bus,'re_ind':retail_ind}}
    
    return dict_factor


# test IV

def select_varia_list(mode):
    if mode =='cor_fi':
        list_variable=['new_DSCR','new_TotalAssetTurnover','new_ReceivableTurnover','new_TotalDebtsToSales',
                    'new_CurrentRatio','new_DERatio','new_InventoryTurnover','new_NetProfitMargin']
    elif mode=='cor_bus':
        list_variable=['new_N101','new_N102','new_N103','new_N104','new_N105','new_N106','new_N107','new_N108','new_N109','new_N110','new_N111',
                      'new_N112','new_N113','new_N114','new_N101','new_N115','new_N116','new_N117']
    elif mode=='cor_ind':
        list_variable=['new_I105','new_I104','new_I103','new_I102','new_I101']
    elif mode=='sme_fi':
        list_variable=['new_DSCR','new_TotalAssetTurnover','new_InventoryTurnover','new_ReceivableTurnover',
                       'new_CurrentRatio','new_TotalDebtsToSales','new_DERatio','new_NetProfitMargin']
    elif mode=='sme_bus':
        list_variable=['new_EE101','new_EE102','new_EE103','new_EE104','new_EE201','new_EE202','new_EE203','new_EXX6','new_EE204','new_EE301',
                       'new_EE302']
    elif mode=='sme_ind':
        list_variable=['new_I105','new_I104','new_I103','new_I102','new_I101']
    elif mode=='re_fi':
        list_variable=['NetProfitMargin_Sc', 'DERatio_Sc', 'TotalDebtsToSales_Sc' ,'CurrentRatio_Sc']
    elif mode=='re_bus':
        list_variable=['B610_Sc','B710_Sc','B720_Sc','new_NCB7','new_NCB4','BusinessAge_Sc','B2010_Sc','CwsFactor1_Sc']
    elif mode=='re_ind':
        list_variable=['new_I105','new_I104','new_I103','new_I102','new_I101']
    return list_variable

def IV_test_factor(df, variable): 
    
    # 1. Drop rows where flag is 1
    df = df.drop(df[df['ExistingDefaultFlag'] == 1].index)
    
    # 2. Check if the required columns exist
    required_columns = ['DefaultFlag12M', 'RequestID', variable]
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in the dataframe.")
    
    # 3. Create 'good_bad' column
    df['good_bad'] = np.where((df['DefaultFlag12M'] == 0), 'good', 'bad')
    
    # 4. Create pivot table by 'CustomerRefID'
    try:
        good_bad_tables = pd.pivot_table(df, values='CustomerRefID', index=[variable], columns='good_bad', aggfunc='count', fill_value=0)
    except Exception as e:
        print(f"Error in creating pivot table: {e}")
        return None, None
    
    # Check if 'good' and 'bad' columns exist in the pivot table
    if 'bad' not in good_bad_tables or 'good' not in good_bad_tables:
        return good_bad_tables, good_bad_tables

    table = good_bad_tables.copy()

    table.sort_values(variable,ascending=True,inplace=True)
    
    # Calculate ROC KS
    table['cumugood'] = table['good'].cumsum() / table['good'].sum()
    table['cumubad'] = table['bad'].cumsum() / table['bad'].sum()
    table['%bad']= 100*table['bad']/(table['good']+table['bad'])
    
    
    table['ROC'] = (table['cumugood'] - table['cumugood'].shift(1, fill_value = 0)) * \
                (table['cumubad'] + table['cumubad'].shift(1, fill_value = 0)) * 0.5
    table['KS'] = abs(table['cumugood'] - table['cumubad'])
    table['AUC (sum ROC)'] = table['ROC'].sum()
    table['AR'] = (table['AUC (sum ROC)'] * 2) - 1
    
    # Create 'Distribution'
    table['Distribution bad (DB)'] = (table['bad'] / table['bad'].sum())
    table['Distribution good (GB)'] = (table['good'] / table['good'].sum())

    # Calculate WOE IV
    table['WOE'] = np.log(table['Distribution good (GB)'] / table['Distribution bad (DB)'])
    table['DG - DB'] = table['Distribution good (GB)'] - table['Distribution bad (DB)']
    table['IV'] = (table['DG - DB'] * table['WOE'])
    table['sum_IV'] = table[table['IV'] != math.inf]['IV'].sum()
    
    # Calculate KS
    table['max_KS']=max(table['KS'] )
    max_KS=max(table['KS'] )
    totalbad=table['bad'].sum()
    totalgood=table['good'].sum()
    # sqrt((m+n)/(m*n))
    z=np.sqrt((totalbad+totalgood)/(totalbad*totalgood))
    critical_D_05 = z*1.36
    critical_D_01 =z*1.22
    table['Confidential Level 90'] =np.where(max_KS > critical_D_01, 'Reject H0','Fail to Reject H0' )
    table['Confidential Level 95'] =np.where(max_KS > critical_D_05, 'Reject H0','Fail to Reject H0')
    table['Order'] = table['%bad'].is_monotonic_decreasing

    table.drop(columns=['Distribution bad (DB)','Distribution good (GB)','DG - DB'],inplace=True)
    count_df=table['good'].sum()+table['bad'].sum()
    return table, good_bad_tables,count_df
   


def run_factor_level_test(dict_factor,folder_path,filter):   
    for port,innerport in dict_factor.items():
        print('factor level',port,filter)
        for name,df in innerport.items():
            output_path=os.path.join(folder_path,'Output','IV_test',port)
            file_path=os.path.join(output_path,f'Cust{filter}_{port}_IV_KS_AUC_result_{name}.xlsx')
            good_bad_dict={}
            count_dict={}
            list_variable=select_varia_list(name)
            first_row=[]
            with pd.ExcelWriter(file_path) as writer:
                columns_list=df.columns.to_list()
                for variable in columns_list :
                    if variable in list_variable:
                        pivot_table,good_bad_tables,count_df=IV_test_factor(df,variable)
                        if pivot_table is not None:
                            styled_df=Apply_style(pivot_table)
                            styled_df.to_excel(writer, sheet_name=variable)
                            pivot_table['factor']=variable
                            first_row.append(pivot_table.iloc[:1])
                            good_bad_dict[variable]=good_bad_tables
                            count_dict[variable]=count_df
                
            file_path=os.path.join(output_path,f'Cust{filter}_{port}_IV_KS_AUC_summary_{name}.xlsx')
            first_row_df=pd.concat(first_row)
            first_row_df.reset_index(inplace=True)
            first_row_df=Apply_style(first_row_df)
            first_row_df.to_excel(file_path)

            output_path=os.path.join(folder_path,'Output','Homogeneity_test',port)
            file_path=os.path.join(output_path,f'Cust{filter}_{port}_homo_result_{name}.xlsx')
            with pd.ExcelWriter(file_path) as writer:
                columns_list=df.columns.to_list()
                for var,goodbad in good_bad_dict.items() :
                    if var in list_variable:
                        pivot_table=Homogeneity(goodbad)
                        if pivot_table is not None:
                            pivot_table.reset_index().to_excel(writer, sheet_name=var, index=False)
                

            output_path=os.path.join(folder_path,'Output','PSI_test',port)
            if not os.path.isdir(output_path):
                os.makedirs(output_path)
            file_path=os.path.join(output_path,f'Cust{filter}_{port}_PSI_result_{name}.xlsx')
            last_row=[]
            with pd.ExcelWriter(file_path) as writer:
                columns_list=df.columns.to_list()
                for variable in columns_list :
                    if variable in list_variable:
                        psi_df=PSI_test(df,variable)
                        if psi_df is not None:
                            styled_df = psi_df.style.map(highlight_PSI, subset=['PSI'])
                            styled_df.to_excel(writer, sheet_name=variable, index=False)
                            psi_df['factor']=variable
                            last_row.append(psi_df.iloc[-1:])   
                
            file_path=os.path.join(output_path,f'Cust{filter}_{port}_PSI_summary_{name}.xlsx')
            last_row_df=pd.concat(last_row)
            last_row_df.reset_index(inplace=True)
            styled_df = last_row_df.style.map(highlight_PSI, subset=['PSI'])
            styled_df.to_excel(file_path)

def highlight_KS(val):
    if val > 0.5:
        color = 'green'
    elif 0.4 < val <= 0.5:
        color = 'yellow'
    elif 0.2 < val <= 0.4:
        color = 'orange'
    else:
        color = 'red'
    text_color = 'white' if color == 'red' else 'black'
    return f'background-color: {color}; color: {text_color}'

def highlight_IV(val):
    if val > 0.3:
        color = 'green'
    elif 0.1 < val <= 0.3:
        color = 'yellow'
    else:
        color = 'red'
    text_color = 'white' if color == 'red' else 'black'
    return f'background-color: {color}; color: {text_color}'

def highlight_AUC(val):
    if val >= 0.8:
        color = 'green'
    elif 0.7 <= val < 0.8:
        color = 'yellow'
    elif 0.6 <= val < 0.7:
        color = 'orange'
    else:
        color = 'red'
    text_color = 'white' if color == 'red' else 'black'
    return f'background-color: {color}; color: {text_color}'

def highlight_AR(val):
    if val >= 0.6:
        color = 'green'
    elif 0.4 <= val < 0.6:
        color = 'yellow'
    elif 0.2 <= val < 0.4:
        color = 'orange'
    else:
        color = 'red'
    text_color = 'white' if color == 'red' else 'black'
    return f'background-color: {color}; color: {text_color}'

def highlight_PSI(val):
    if val >= 0.25:
        color = 'red'
    elif 0.1 <= val < 0.25:
        color = 'yellow'
    elif val < 0.1:
        color = 'green'
    text_color = 'white' if color == 'red' else 'black'
    return f'background-color: {color}; color: {text_color}'



def Apply_style(df):
    styled_df = df.style.map(highlight_KS, subset=['max_KS'])
    styled_df = styled_df.map(highlight_IV, subset=['sum_IV'])
    styled_df = styled_df.map(highlight_AUC, subset=['AUC (sum ROC)'])
    styled_df = styled_df.map(highlight_AR, subset=['AR'])
    return styled_df

# PSI
def population_stability_index(dev_data, val_data,col_name1,col_name2):
    dev_group = dev_data.groupby(col_name2).count()
    dev_group.rename(columns={col_name1: 'dev_count'} , inplace = True)
    val_group = val_data.groupby(col_name2).count()
    val_group.rename(columns={col_name1: 'val_count'} , inplace = True)

    merged_counts = dev_group.merge(val_group, on=col_name2, how='left')
    merged_counts['dev_pct'] = (merged_counts['dev_count'] / merged_counts['dev_count'].sum())
    merged_counts['val_pct'] = (merged_counts['val_count'] / merged_counts['val_count'].sum())
    merged_counts['psi'] = (merged_counts['val_pct'] - merged_counts['dev_pct']) * np.log(merged_counts['val_pct'] / merged_counts['dev_pct'])
    return merged_counts['psi'].sum()

def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

def update_dictionary(key, val, test_dict):
    if key not in test_dict:
        test_dict[key] = val 
    elif type(val) not in [str, list, dict]:
        raise ValueError("Invalid Data Type")
    elif isinstance(val, list):
        test_dict[key].extend(val)
    elif isinstance(val, dict):
        test_dict[key].update(val)
    else:
        test_dict[key] = test_dict[key] + val
    return test_dict

def PSI_test(corp_df,column_name):
    dev_month = datetime.date(2023, 9, 30) + MonthEnd(0)
    dev_data = corp_df[(corp_df['RatingDate'] <= dev_month)]
    val_data_raw = corp_df
    latest_month = max(val_data_raw['RatingDate']) + MonthEnd(0)
    no_month = diff_month(latest_month, dev_month)

    def PSI_result(no_month,dev_month,val_data_raw,dev_data,column_name):
        dict={'date':[],'PSI':[]}
        for x in range(1,no_month+1):
            end_date = dev_month + MonthEnd(x)
            test = val_data_raw[(val_data_raw['RatingDate'] <= end_date)]
            val_data = test[['RequestID',column_name]]
            psi_value = population_stability_index(dev_data, val_data, 'RequestID', column_name)
            date,psi=[end_date],[psi_value]
            update_dictionary('date', date,dict)
            update_dictionary('PSI', psi,dict)
        psi_df=pd.DataFrame(dict)
        return psi_df
    
    psi_df=PSI_result(no_month,dev_month,val_data_raw,dev_data,column_name)

    return psi_df

# import data

In [ ]:
# output folder
folder_path= "../"

#import fraud account
df_fraud_path="../data/raw/9.-12. ข้อมูลที่ใช้ในการทดสอบ_Oct 2019 - Dec 2022.xlsx"
fraud_list=Fraud_import(df_fraud_path)

#import Corporate
path="../data/processed/02_data_sampling/corporate_customer_data.parquet"
df_corporate=pd.read_parquet(path)
df_corporate_filtered=fraud_filter(df_corporate,fraud_list)

#import SME
sme_path="../data/processed/02_data_sampling/sme_customer_data.parquet"
df_sme=pd.read_parquet(sme_path)
df_sme_filtered=fraud_filter(df_sme,fraud_list)

#import Retail
retail_path="../data/processed/02_data_sampling/Retail_202005_202403.parquet"
df_retail=pd.read_parquet(retail_path)
df_retail_filtered=fraud_filter(df_retail,fraud_list)


# import factor
bus_new_df,bus_old_df= import_factor("../data/processed/01_master_data/ScoreResult_Business_Dec22_Mar24.parquet",
                                   "../data/processed/01_master_data/ScoreResult_Business_old_Nov22.parquet")

bussme_new_df,bussme_old_df =import_factor("../data/processed/01_master_data/ScoreResult_BusinessSMEs_Dec22_Mar24.parquet",
                                         "../data/processed/01_master_data/ScoreResult_Express_old_Nov22.parquet")

ind_new_df,ind_old_df =import_factor("../data/processed/01_master_data/ScoreResult_Industry_Dec22_Mar24.parquet",
                                   "../data/processed/01_master_data/ScoreResult_Industry_old_Nov22.parquet")

fi_new_df,fi_old_df=import_factor("../data/processed/01_master_data//ScoreResult_Financial_Dec22_Mar24.parquet",
                                  "../data/processed/01_master_data//ScoreResult_Financial_old_Nov22.parquet")

BX17_df =import_BX17("../data/raw")

# run python

In [ ]:
# composite level
merge_dict_customer={}
merge_dict_customer['Composite']=pd.concat([df_corporate,df_sme,df_retail])
merge_dict_customer_filtered={}
merge_dict_customer_filtered['Composite']=pd.concat([df_corporate_filtered,df_sme_filtered,df_retail_filtered])

#change_criteria
df_com_customer=change_criteria(merge_dict_customer)
df_com_customer_filtered=change_criteria(merge_dict_customer_filtered)

# composite level
all_test(df_com_customer,'Cust',folder_path)
all_test(df_com_customer_filtered,'Cust_filter',folder_path)

#factor level
cor_fi,cor_bus,cor_in,sme_fi,sme_bus,sme_ind,retail_fi,retail_bus,retail_ind=factor_prepare_factor_df(df_corporate,df_sme,df_retail,fi_new_df,fi_old_df,bus_new_df,bus_old_df,bussme_new_df,bussme_old_df,ind_new_df,ind_old_df,BX17_df)
#binning factor
dict_factor=run_all_factor (cor_fi,cor_bus,cor_in,sme_fi,sme_bus,sme_ind,retail_fi,retail_bus,retail_ind)
#factor test
run_factor_level_test(dict_factor,folder_path,'')

#prepare filter df
cor_fi,cor_bus,cor_in,sme_fi,sme_bus,sme_ind,retail_fi,retail_bus,retail_ind=factor_prepare_factor_df(df_corporate_filtered,df_sme_filtered,df_retail_filtered,fi_new_df,fi_old_df,bus_new_df,bus_old_df,bussme_new_df,bussme_old_df,ind_new_df,ind_old_df,BX17_df)
#binning factor
dict_factor=run_all_factor (cor_fi,cor_bus,cor_in,sme_fi,sme_bus,sme_ind,retail_fi,retail_bus,retail_ind)
#factor test
run_factor_level_test(dict_factor,folder_path,'_filter')

print('Finish')
